# Heart Disease Prediction — TenYearCHD (D11KS)

## Mô tả bài toán 

Dataset: `D11KS.csv` (**11,000 dòng × 16 cột**)

Mục tiêu: dự đoán nhãn **`TenYearCHD`**:
- `0`: không có nguy cơ bệnh mạch vành trong 10 năm tới
- `1`: có nguy cơ bệnh mạch vành trong 10 năm tới

Đây là **bài toán phân loại nhị phân (binary classification)**.

### I Tình trạng dữ liệu (để bạn đưa vào báo cáo)
- Sau khi bỏ dòng thiếu target: lớp 0 ≈ **9,006**, lớp 1 ≈ **1,633**
- Tỷ lệ minor/major ≈ **0.181** ⇒ dữ liệu **mất cân bằng**

### II Pipeline — tránh data leakage
1) Load + kiểm tra cơ bản  
2) Cleaning (không học thống kê): drop target NaN, mapping Yes/No & gender, ép numeric, xử lý hút thuốc không nhất quán, drop duplicates  
3) EDA (Plotly): imbalance, missing, histogram/boxplot (gộp), correlation  
4) Split train/test (stratify)  
5) Downsample **nhãn 0 chỉ trên TRAIN**  
6) Preprocess theo model:
   - **Linear (Lo.R, So.R):** Impute + **StandardScaler**
   - **Tree/Boosting (De.T, Ra.F, Ad.B, Gr.B, XG.B, Li.G):** Impute (không cần scaler)
7) Train & đánh giá **Train / Val / Test**  
   - Chỉ **vẽ ROC** (các metric khác hiển thị dạng bảng)

---

## III Từ điển feature (ý nghĩa + range trong dataset)

| Feature | Kiểu | Ý nghĩa | Giá trị / Range (theo data) | Missing |
|---|---|---|---|---|
| `gender` | categorical | Giới tính sinh học (Male/Female). | Male, Female | 0.51% |
| `age` | numeric | Tuổi (năm). | 32 → 70 | 0.68% |
| `education` | numeric | Trình độ học vấn (mã hoá 1–4, dạng thứ bậc/ordinal). | 1 → 4 | 0.00% |
| `currentSmoker` | categorical | Hiện tại có hút thuốc không (Yes/No). | No, Yes | 0.00% |
| `cigsPerDay` | numeric | Số điếu thuốc hút mỗi ngày. | 0 → 70 | 0.42% |
| `BPMeds` | categorical | Đang dùng thuốc huyết áp (Blood Pressure Meds) (Yes/No). | No, Yes | 0.00% |
| `prevalentStroke` | categorical | Tiền sử đột quỵ (Yes/No). | No, Yes | 0.00% |
| `prevalentHyp` | categorical | Tăng huyết áp (Yes/No). | No, Yes | 1.17% |
| `diabetes` | categorical | Tiểu đường (Yes/No). | No, Yes | 1.97% |
| `totChol` | numeric | Cholesterol toàn phần (thường mg/dL). | 107 → 696 | 0.36% |
| `sysBP` | numeric | Huyết áp tâm thu (thường mmHg). | 83.5 → 295 | 0.00% |
| `diaBP` | numeric | Huyết áp tâm trương (thường mmHg). | 48 → 142.5 | 3.75% |
| `BMI` | numeric | Body Mass Index (kg/m²). | 15.54 → 56.8 | 0.59% |
| `heartRate` | numeric | Nhịp tim (beats per minute). | 44 → 143 | 0.44% |
| `glucose` | numeric | Đường huyết (thường mg/dL). | 40 → 394 | 0.00% |
| `TenYearCHD` | numeric | Nhãn: nguy cơ bệnh mạch vành trong 10 năm (0/1). | 0 → 1 | 3.28% |



In [2]:
# 0) SETUP: imports, seed, warnings, optional libs (KHÔNG pip install trong notebook)

import os
import warnings
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt  # fallback nếu thiếu plotly
from IPython.display import display


from sklearn.model_selection import (
    train_test_split, StratifiedKFold, cross_validate, RandomizedSearchCV, cross_val_predict
)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline as SkPipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    confusion_matrix, classification_report, brier_score_loss,
    precision_recall_curve
)
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

warnings.filterwarnings("ignore")

SEED = 42
np.random.seed(SEED)

# ---- Optional libs flags ----
HAS_SEABORN, HAS_IMBLEARN, HAS_PLOTLY, HAS_XGB, HAS_LGBM = False, False, False, False, False

# seaborn (không bắt buộc)
try:
    import seaborn as sns
    HAS_SEABORN = True
except Exception:
    print("seaborn not installed -> fallback matplotlib")

# imbalanced-learn (sampler/pipeline)
try:
    from imblearn.pipeline import Pipeline as ImbPipeline
    from imblearn.over_sampling import RandomOverSampler
    HAS_IMBLEARN = True
except Exception:
    print("imbalanced-learn not installed -> no resampling (use class_weight where possible)")

# plotly (EDA + plots)
try:
    import plotly.express as px
    import plotly.graph_objects as go
    import plotly.io as pio
    HAS_PLOTLY = True
    try:
        pio.renderers.default = "notebook_connected"
    except Exception:
        pass
except Exception:
    print("plotly not installed -> fallback matplotlib")

# xgboost
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except Exception:
    print("xgboost not installed -> XGB skipped")

# lightgbm
try:
    from lightgbm import LGBMClassifier
    HAS_LGBM = True
except Exception:
    print("lightgbm not installed -> LGBM skipped")


In [3]:
# 1) LOAD DATA

DATA_PATH = "Data/D11KS.csv"
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(
        f"Không tìm thấy '{DATA_PATH}'. Hãy kiểm tra lại đường dẫn hoặc đặt file đúng thư mục."
    )

df = pd.read_csv(DATA_PATH)
display(df.head())
print("Shape:", df.shape)


,gender,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,Male,39.0,4.0,No,0.0,No,No,No,No,195.0,106.0,70.0,26.97,80.0,77.0,0.0
1,Female,46.0,2.0,No,0.0,No,No,No,No,250.0,121.0,81.0,28.73,95.0,76.0,0.0
2,Male,48.0,1.0,Yes,20.0,No,No,No,No,245.0,127.5,80.0,25.34,75.0,70.0,0.0
3,Female,61.0,3.0,Yes,30.0,No,No,Yes,No,225.0,150.0,95.0,28.58,65.0,103.0,1.0
4,Female,46.0,3.0,Yes,23.0,No,No,No,No,285.0,130.0,84.0,23.10,85.0,85.0,0.0


Shape: (11000, 16)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   gender           10944 non-null  object 
 1   age              10925 non-null  float64
 2   education        11000 non-null  float64
 3   currentSmoker    11000 non-null  object 
 4   cigsPerDay       10954 non-null  float64
 5   BPMeds           11000 non-null  object 
 6   prevalentStroke  11000 non-null  object 
 7   prevalentHyp     10871 non-null  object 
 8   diabetes         10783 non-null  object 
 9   totChol          10960 non-null  float64
 10  sysBP            11000 non-null  float64
 11  diaBP            10588 non-null  float64
 12  BMI              10935 non-null  float64
 13  heartRate        10952 non-null  float64
 14  glucose          11000 non-null  float64
 15  TenYearCHD       10639 non-null  float64
dtypes: float64(10), object(6)
memory usage: 1.3+ MB


** Kiểm tra cấu trúc dữ liệu (columns, dtypes) và cảnh báo thiếu cột kỳ vọng**

Cell này giúp bạn kiểm tra nhanh dataset sau khi load:

- In ra danh sách **tên cột** hiện có trong `df`.
- Hiển thị **kiểu dữ liệu (dtype)** của từng cột để phát hiện sai kiểu (vd: số nhưng đang là object).
- Định nghĩa danh sách `expected_cols` là các cột “chuẩn” mong đợi cho bài toán CHD 10 năm.
- So sánh để tìm các cột bị thiếu và in cảnh báo nếu dataset không đủ cột cần thiết.


In [5]:
# Kiểm tra nhanh cấu trúc dataset: danh sách cột, dtype từng cột, và cảnh báo nếu thiếu cột quan trọng

# In danh sách các cột hiện có
print("Columns:", list(df.columns))

# Hiển thị kiểu dữ liệu của từng cột (giúp phát hiện cột số bị đọc thành object/string)
print("\nDtypes:")
display(df.dtypes.to_frame("dtype"))

# Danh sách các cột kỳ vọng (schema chuẩn) cho bài toán dự đoán TenYearCHD
expected_cols = [
    "gender","age","education","currentSmoker","cigsPerDay","BPMeds","prevalentStroke",
    "prevalentHyp","diabetes","totChol","sysBP","diaBP","BMI","heartRate","glucose","TenYearCHD"
]

# Tìm các cột bị thiếu so với schema kỳ vọng
missing = [c for c in expected_cols if c not in df.columns]

# Nếu thiếu thì cảnh báo để bạn biết dataset không đúng cấu trúc mong đợi
if missing:
    print("WARNING: Missing expected columns:", missing)


Columns: ['gender', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose', 'TenYearCHD']

Dtypes:


,dtype
gender,object
age,float64
education,float64
currentSmoker,object
cigsPerDay,float64
BPMeds,object
prevalentStroke,object
prevalentHyp,object
diabetes,object
totChol,float64


In [6]:
# Missing values: đếm số lượng NaN theo từng cột
miss_cnt = df.isna().sum()

# Missing values: tính % NaN theo từng cột
miss_pct = (miss_cnt / len(df) * 100).round(2)

# Tạo bảng tổng hợp missing và sắp xếp giảm dần theo % missing
missing_tbl = (
    pd.DataFrame({"missing_count": miss_cnt, "missing_pct": miss_pct})
      .sort_values("missing_pct", ascending=False)
)

display(missing_tbl)


,missing_count,missing_pct
diaBP,412,3.75
TenYearCHD,361,3.28
diabetes,217,1.97
prevalentHyp,129,1.17
age,75,0.68
BMI,65,0.59
gender,56,0.51
heartRate,48,0.44
cigsPerDay,46,0.42
totChol,40,0.36


In [7]:
# Target distribution: đếm số lượng mỗi lớp (bao gồm cả NaN để kiểm tra dữ liệu bẩn)
y_counts = df["TenYearCHD"].value_counts(dropna=False)

# Target distribution: tính % mỗi lớp
y_pct = (y_counts / y_counts.sum() * 100).round(2)

display(pd.DataFrame({"count": y_counts, "pct": y_pct}))


,count,pct
TenYearCHD,,
0.0,9006,81.87
1.0,1633,14.85
NaN,361,3.28


**Chuẩn hoá giá trị categorical + ép kiểu numeric**

Cell này chuẩn hoá các cột dạng text/categorical và ép kiểu các cột số để dữ liệu nhất quán trước khi EDA & modeling.

- `normalize_gender`: chuẩn hoá các biến thể của giới tính (m/male/nam/1 → `male`, f/female/nữ/0 → `female`), giá trị không hợp lệ → `NaN`.
- `normalize_yesno`: chuẩn hoá các biến thể Yes/No (yes/true/1 → `Yes`, no/false/0 → `No`), giá trị không hợp lệ → `NaN`.
- Với các cột trong `TEXT_COLS`, áp dụng hàm chuẩn hoá và chuyển sang kiểu `category`.
- Với các cột trong `NUMERIC_COLS`, ép kiểu về số bằng `pd.to_numeric(errors="coerce")` để chuyển giá trị lỗi sang `NaN`.


In [8]:
# Chuẩn hoá categorical (gender, yes/no) và ép kiểu numeric để dữ liệu sạch & đồng nhất trước modeling

def normalize_gender(x):
    # Chuẩn hoá cột gender về 2 nhãn: 'male' / 'female'; giá trị lạ -> NaN
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    if s in {"m", "male", "nam", "1"}:
        return "male"
    if s in {"f", "female", "nu", "nữ", "0"}:
        return "female"
    return np.nan

def normalize_yesno(x):
    # Chuẩn hoá các cột dạng yes/no về 'Yes' / 'No'; giá trị lạ -> NaN
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    if s in {"yes", "y", "true", "1"}:
        return "Yes"
    if s in {"no", "n", "false", "0"}:
        return "No"
    return np.nan

# Các cột text/categorical cần chuẩn hoá (nếu có trong df)
TEXT_COLS = ["gender", "currentSmoker", "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes"]
for c in TEXT_COLS:
    # Bỏ qua nếu dataset không có cột đó
    if c not in df.columns:
        continue

    # gender dùng rule riêng; các cột còn lại dùng normalize_yesno
    if c == "gender":
        df[c] = df[c].apply(normalize_gender).astype("category")
    else:
        df[c] = df[c].apply(normalize_yesno).astype("category")

# Các cột numeric: ép về số, giá trị không parse được sẽ thành NaN
NUMERIC_COLS = ["age", "cigsPerDay", "totChol", "sysBP", "diaBP", "BMI", "heartRate", "glucose"]
for c in NUMERIC_COLS:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")


## EDA (mô tả dữ liệu)
> EDA chỉ để mô tả, **không** dùng để fit scaler/encoder.

In [9]:
# (1) Bảng tổng quan: xem nhanh shape + dtype + số giá trị unique + số missing theo từng cột

# In kích thước dataset (số dòng, số cột)
print("df.shape =", df.shape)

# Tạo bảng overview:
# - dtype: kiểu dữ liệu của cột
# - n_unique: số lượng giá trị khác nhau (kể cả NaN)
# - n_missing: số lượng giá trị bị thiếu
overview = pd.DataFrame({
    "dtype": df.dtypes.astype(str),
    "n_unique": df.nunique(dropna=False),
    "n_missing": df.isna().sum()
}).sort_values("n_missing", ascending=False)  # sắp xếp để cột thiếu nhiều nằm trên

display(overview)


df.shape = (11000, 16)


,dtype,n_unique,n_missing
diaBP,float64,679,412
TenYearCHD,float64,3,361
diabetes,category,3,217
prevalentHyp,category,3,129
age,float64,40,75
BMI,float64,1885,65
gender,category,3,56
heartRate,float64,83,48
cigsPerDay,float64,53,46
totChol,float64,285,40


In [10]:
# (2) Bảng missing: thống kê số lượng (count) và tỷ lệ (%) giá trị thiếu theo từng cột

# Tạo bảng missing gồm:
# - missing_count: tổng số NaN của mỗi cột
# - missing_pct: % NaN của mỗi cột (mean của isna() * 100)
missing_tbl = pd.DataFrame({
    "missing_count": df.isna().sum(),
    "missing_pct": (df.isna().mean() * 100).round(2)
}).sort_values("missing_pct", ascending=False)  # sắp xếp giảm dần theo % missing

display(missing_tbl)


,missing_count,missing_pct
diaBP,412,3.75
TenYearCHD,361,3.28
diabetes,217,1.97
prevalentHyp,129,1.17
age,75,0.68
BMI,65,0.59
gender,56,0.51
heartRate,48,0.44
cigsPerDay,46,0.42
totChol,40,0.36


In [11]:
# (3) Phân phối target (bar + %) — thống kê count/% và vẽ bar chart bằng Plotly

# Ép TenYearCHD về numeric để xử lý trường hợp dữ liệu bị đọc sai kiểu (string, ký tự lạ)
y = pd.to_numeric(df["TenYearCHD"], errors="coerce")

# Loại bỏ giá trị không hợp lệ (inf/-inf) và drop NaN để tránh lỗi khi ép int
y = y.replace([np.inf, -np.inf], np.nan).dropna().astype(int)

# (Tuỳ chọn) Nếu muốn chắc chắn target chỉ là nhị phân 0/1 thì bật dòng dưới
# y = y[y.isin([0, 1])]

# Đếm số lượng mỗi lớp (0/1) và sort theo index để hiển thị theo thứ tự 0 -> 1
counts = y.value_counts().sort_index()

# Tính % theo từng lớp
pct = (counts / counts.sum() * 100).round(2)

# Hiển thị bảng count + percent
display(pd.DataFrame({"count": counts, "percent": pct}))

# Chuẩn bị dataframe cho Plotly (đổi TenYearCHD sang string để trục X hiển thị dạng category)
plot_df = pd.DataFrame({
    "TenYearCHD": counts.index.astype(str),
    "count": counts.values,
    "percent": pct.values
})

# Vẽ bar chart: trục Y là count, label trên cột là percent
fig = px.bar(
    plot_df,
    x="TenYearCHD",
    y="count",
    text="percent",
    title="Target distribution (count + %)"
)

# Format label hiển thị: gắn dấu % và đặt text phía trên cột
fig.update_traces(texttemplate="%{text:.2f}%", textposition="outside")

# Set title trục + tăng y-range để chừa chỗ cho label phía trên
fig.update_layout(
    xaxis_title="TenYearCHD",
    yaxis_title="Count",
    yaxis_range=[0, plot_df["count"].max() * 1.15]
)

fig.show()


,count,percent
TenYearCHD,,
0,9006,84.65
1,1633,15.35


**(4) Histogram cho `age` theo `TenYearCHD` (vẽ gì và đọc như nào?)**

Biểu đồ này đang **so sánh phân phối (distribution)** của biến `age` giữa các nhóm `TenYearCHD`:

- `TenYearCHD = 0` (không mắc CHD trong 10 năm)
- `TenYearCHD = 1` (mắc CHD trong 10 năm)

Cụ thể, nó vẽ **histogram (cột)** của `age` cho từng nhóm và **chồng lên nhau (overlay)** để bạn nhìn xem nhóm nào “lệch” sang tuổi cao hơn/thấp hơn.
Khi bạn dùng `histnorm="probability density"` thì trục Y là **mật độ (density)** → giúp so sánh hình dạng phân phối ngay cả khi 2 nhóm có số lượng khác nhau.

**Vì sao bạn thấy 3 distribution?**
Thường là do `TenYearCHD` có thêm giá trị khác `0/1` (ví dụ: NaN, 2, hoặc string lạ). Cách xử lý là ép về numeric + lọc chỉ giữ {0,1}.


In [12]:
# (4) Histogram/KDE cho age theo target — so sánh phân phối age giữa nhóm CHD=0 và CHD=1

feat = "age"

# Lấy 2 cột cần thiết và loại bỏ missing để plot không bị lỗi
tmp = df[[feat, "TenYearCHD"]].copy()
tmp["TenYearCHD"] = pd.to_numeric(tmp["TenYearCHD"], errors="coerce").replace([np.inf, -np.inf], np.nan)
tmp = tmp.dropna(subset=[feat, "TenYearCHD"])

# Ép target về int rồi đổi sang string để Plotly coi là category khi tô màu
tmp["TenYearCHD"] = tmp["TenYearCHD"].astype(int).astype(str)

if HAS_PLOTLY:
    # Plotly histogram dạng overlay, chuẩn hoá về density để so sánh hình dạng phân phối
    fig = px.histogram(
        tmp,
        x=feat,
        color="TenYearCHD",
        nbins=30,
        barmode="overlay",
        histnorm="probability density",
        title=f"Distribution of {feat} by TenYearCHD"
    )
    fig.update_layout(xaxis_title=feat, yaxis_title="Density")
    fig.show()
else:
    # Fallback Matplotlib: vẽ histogram density cho từng lớp
    fig, ax = plt.subplots()
    for cls in ["0", "1"]:
        data = tmp.loc[tmp["TenYearCHD"] == cls, feat]
        ax.hist(data, bins=30, alpha=0.5, density=True, label=f"CHD={cls}")

    ax.set_title(f"Distribution of {feat} by TenYearCHD")
    ax.set_xlabel(feat)
    ax.set_ylabel("Density")
    ax.legend()
    plt.show()


**(5) Boxplot phát hiện outlier cho các biến số (theo từng lớp TenYearCHD)**

Cell này vẽ **boxplot** cho một nhóm biến numeric quan trọng (BMI, huyết áp, cholesterol, glucose, …) để:

- Quan sát **median** (đường giữa hộp), **IQR** (chiều cao hộp), và mức độ **phân tán** của dữ liệu.
- Phát hiện **outlier** (các điểm nằm ngoài “râu” boxplot).
- So sánh phân phối của từng biến giữa 2 nhóm `TenYearCHD = 0` và `TenYearCHD = 1` thông qua màu sắc.

Cách hoạt động:
- Chọn các cột trong `box_cols` (chỉ lấy những cột thật sự tồn tại).
- Làm sạch `TenYearCHD` để tránh NaN/inf gây lỗi khi ép kiểu.
- Chuyển dữ liệu sang dạng “long format” bằng `melt` để Plotly có thể vẽ nhiều boxplot (mỗi feature là 1 nhóm trên trục X).
- Dùng `points="outliers"` để hiển thị các điểm outlier.


In [13]:
# (5) Boxplot phát hiện outlier (Plotly): so sánh phân phối các biến numeric theo nhóm TenYearCHD

# Danh sách các biến muốn kiểm tra outlier (chỉ lấy cột tồn tại trong df)
box_cols = ["BMI", "sysBP", "diaBP", "totChol", "glucose", "heartRate", "cigsPerDay"]
box_cols = [c for c in box_cols if c in df.columns]

# Lấy subset dữ liệu gồm các biến numeric + target
tmp = df[box_cols + ["TenYearCHD"]].copy()

# Làm sạch target TenYearCHD:
# - ép về numeric để loại bỏ giá trị string/lỗi
# - thay inf/-inf bằng NaN
# - drop các dòng target NaN để tránh lỗi khi ép int
tmp["TenYearCHD"] = pd.to_numeric(tmp["TenYearCHD"], errors="coerce")
tmp["TenYearCHD"] = tmp["TenYearCHD"].replace([np.inf, -np.inf], np.nan)
tmp = tmp.dropna(subset=["TenYearCHD"])

# Ép target về int rồi chuyển sang str để Plotly coi là category khi tô màu
tmp["TenYearCHD"] = tmp["TenYearCHD"].astype(int).astype(str)

# Chuyển dữ liệu sang dạng long: mỗi dòng là (TenYearCHD, feature, value)
# -> Plotly sẽ vẽ boxplot theo feature trên trục X và color theo TenYearCHD
long_df = tmp.melt(id_vars="TenYearCHD", var_name="feature", value_name="value").dropna()

# Vẽ boxplot + hiển thị các điểm outlier
fig = px.box(
    long_df,
    x="feature",
    y="value",
    color="TenYearCHD",
    points="outliers",
    title="Outlier check (boxplot) for key numeric features"
)

# Chỉnh nhãn trục cho gọn
fig.update_layout(xaxis_title="", yaxis_title="Value")
fig.show()


**(6) Heatmap tương quan các biến numeric (Correlation heatmap) — dùng để làm gì?**

Cell này tính và vẽ **ma trận tương quan (correlation matrix)** giữa các biến số (numeric).  
Tương quan (thường là Pearson) nằm trong khoảng **[-1, 1]**:

- **≈ +1**: 2 biến tăng/giảm cùng nhau rất mạnh (tương quan thuận mạnh)
- **≈ -1**: 1 biến tăng thì biến kia giảm mạnh (tương quan nghịch mạnh)
- **≈ 0**: gần như không có quan hệ tuyến tính

**Vì sao bạn thấy “ít feature”?**  
Vì đoạn code chỉ lấy các cột trong list `num_for_corr` (8 cột). Nếu bạn có nhiều feature numeric hơn, bạn cần:
- tự động lấy tất cả numeric columns, hoặc
- mở rộng list `num_for_corr`.

**Dùng bảng này để chọn feature train được không?**  
Dùng **để tham khảo** chứ không nên chọn feature chỉ dựa vào correlation:
- Correlation chỉ đo **quan hệ tuyến tính giữa các feature** (feature-feature), không trực tiếp nói feature nào “dự đoán tốt target”.
- Nó hữu ích để phát hiện **đa cộng tuyến (multicollinearity)**: nếu 2 feature tương quan quá cao (|corr| > 0.85–0.95), mô hình tuyến tính có thể kém ổn định → cân nhắc bỏ bớt 1 feature hoặc dùng regularization (L1/L2).
- Với tree/boosting, đa cộng tuyến thường ít nghiêm trọng hơn, nhưng vẫn có thể gây “trùng thông tin”.


In [14]:
# (6) Heatmap tương quan numeric (Plotly) — hiển thị dạng hình vuông + dễ nhìn hơn

# Cách 1: tự động lấy TẤT CẢ numeric features (khuyến nghị nếu bạn có nhiều feature)
num_for_corr = df.select_dtypes(include=[np.number]).columns.tolist()

# Nếu muốn loại target ra khỏi heatmap (để chỉ xem feature-feature):
# TARGET = "TenYearCHD"
# num_for_corr = [c for c in num_for_corr if c != TARGET]

# Nếu bạn chỉ muốn giữ list cũ thì comment dòng trên và dùng lại list này:
# num_for_corr = ["age","cigsPerDay","totChol","sysBP","diaBP","BMI","heartRate","glucose"]
# num_for_corr = [c for c in num_for_corr if c in df.columns]

# Tính correlation matrix (Pearson mặc định)
corr = df[num_for_corr].corr()

display(corr.round(3))

if HAS_PLOTLY:
    # Plotly heatmap: làm hình vuông bằng cách set width/height bằng nhau
    fig = px.imshow(
        corr,
        text_auto=".2f",          # hiển thị số với 2 chữ số thập phân
        aspect="equal",           # ép ô vuông (square cells)
        title="Correlation heatmap (numeric features)",
        color_continuous_scale="RdBu_r",  # palette dễ nhìn (đỏ-xanh, đảo chiều)
        zmin=-1, zmax=1           # fix thang màu chuẩn cho correlation
    )
    fig.update_layout(width=850, height=850)  # hình vuông
    fig.show()
else:
    import seaborn as sns
    fig, ax = plt.subplots(figsize=(8, 8))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=True, ax=ax)
    ax.set_title("Correlation heatmap (numeric features)")
    plt.show()


,age,education,cigsPerDay,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
age,1.000,-0.143,-0.187,0.242,0.373,0.203,0.128,-0.004,0.100,0.226
education,-0.143,1.000,0.007,-0.011,-0.112,-0.058,-0.122,-0.066,-0.029,-0.055
cigsPerDay,-0.187,0.007,1.000,-0.026,-0.090,-0.058,-0.095,0.067,-0.049,0.045
totChol,0.242,-0.011,-0.026,1.000,0.188,0.159,0.115,0.091,0.034,0.068
sysBP,0.373,-0.112,-0.090,0.188,1.000,0.753,0.322,0.160,0.106,0.202
diaBP,0.203,-0.058,-0.058,0.159,0.753,1.000,0.378,0.164,0.043,0.135
BMI,0.128,-0.122,-0.095,0.115,0.322,0.378,1.000,0.077,0.083,0.077
heartRate,-0.004,-0.066,0.067,0.091,0.160,0.164,0.077,1.000,0.086,0.013
glucose,0.100,-0.029,-0.049,0.034,0.106,0.043,0.083,0.086,1.000,0.116
TenYearCHD,0.226,-0.055,0.045,0.068,0.202,0.135,0.077,0.013,0.116,1.000


**7. Barplot tỷ lệ TenYearCHD theo categorical (%) (3 bảng: gender, currentSmoker, diabetes)**

Mục tiêu của mục (7) là xem **tỷ lệ mắc CHD trong 10 năm (TenYearCHD = 1)** thay đổi như thế nào theo một số biến phân loại quan trọng.

Cách tính trong cả 3 bảng đều giống nhau:
- Lấy 2 cột: biến categorical `c` và `TenYearCHD`
- Làm sạch `TenYearCHD` (ép numeric, bỏ NaN/inf) rồi ép về **0/1**
- Nhóm theo `c` và tính:  
  **CHD rate (%) = mean(TenYearCHD) × 100**  
  (vì `TenYearCHD` là 0/1 nên giá trị trung bình chính là tỷ lệ dương tính)
- Hiển thị **bảng tỷ lệ** và vẽ **bar chart** (trục Y là % CHD)

Ba biểu đồ/bảng tương ứng:
- **7.1**: Tỷ lệ TenYearCHD theo **gender**
- **7.2**: Tỷ lệ TenYearCHD theo **currentSmoker**
- **7.3**: Tỷ lệ TenYearCHD theo **diabetes**

Diễn giải:
- Cột/bar cao hơn nghĩa là nhóm đó có **tỷ lệ CHD cao hơn** trong dữ liệu.
- Nếu thấy xuất hiện “nhóm lạ” hoặc % bất thường, thường do `TenYearCHD` không chỉ có 0/1 hoặc categorical bị lẫn kiểu (Yes/No trộn 0/1) → cần chuẩn hoá dữ liệu trước.


In [15]:
# 7.1) Barplot tỷ lệ TenYearCHD theo gender (%)

c = "gender"
if c in df.columns:
    tmp = df[[c, "TenYearCHD"]].copy()

    tmp["TenYearCHD"] = pd.to_numeric(tmp["TenYearCHD"], errors="coerce").replace([np.inf, -np.inf], np.nan)
    tmp = tmp.dropna(subset=[c, "TenYearCHD"])
    tmp["TenYearCHD"] = tmp["TenYearCHD"].astype(int)

    rate = (tmp.groupby(c)["TenYearCHD"].mean() * 100).reset_index().rename(columns={"TenYearCHD": "chd_rate_pct"})
    display(rate)

    if HAS_PLOTLY:
        fig = px.bar(rate, x=c, y="chd_rate_pct", text="chd_rate_pct", title=f"TenYearCHD rate by {c} (%)")
        fig.update_traces(texttemplate="%{text:.2f}%", textposition="outside")
        fig.update_layout(yaxis_title="CHD rate (%)")
        fig.show()
    else:
        fig, ax = plt.subplots()
        ax.bar(rate[c].astype(str), rate["chd_rate_pct"])
        ax.set_title(f"TenYearCHD rate by {c} (%)")
        ax.set_ylabel("CHD rate (%)"); ax.set_xlabel(c)
        plt.show()
else:
    print("Column 'gender' not found in df.")


,gender,chd_rate_pct
0,female,12.784185
1,male,18.808152


In [16]:
# 7.2) Tỷ lệ TenYearCHD theo currentSmoker (%)

c = "currentSmoker"
if c in df.columns:
    tmp = df[[c, "TenYearCHD"]].copy()

    # Clean target để tránh lỗi + đảm bảo đúng kiểu 0/1
    tmp["TenYearCHD"] = pd.to_numeric(tmp["TenYearCHD"], errors="coerce").replace([np.inf, -np.inf], np.nan)
    tmp = tmp.dropna(subset=[c, "TenYearCHD"])
    tmp["TenYearCHD"] = tmp["TenYearCHD"].astype(int)

    # mean(target) * 100 = % CHD trong từng nhóm currentSmoker
    rate = (tmp.groupby(c, dropna=False)["TenYearCHD"].mean() * 100).reset_index()
    rate = rate.rename(columns={"TenYearCHD": "chd_rate_pct"})
    display(rate)

    if HAS_PLOTLY:
        fig = px.bar(rate, x=c, y="chd_rate_pct", text="chd_rate_pct",
                     title=f"TenYearCHD rate by {c} (%)")
        fig.update_traces(texttemplate="%{text:.2f}%", textposition="outside")
        fig.update_layout(yaxis_title="CHD rate (%)", xaxis_title=c)
        fig.show()
    else:
        fig, ax = plt.subplots()
        ax.bar(rate[c].astype(str), rate["chd_rate_pct"])
        ax.set_title(f"TenYearCHD rate by {c} (%)")
        ax.set_ylabel("CHD rate (%)"); ax.set_xlabel(c)
        plt.show()
else:
    print("Column 'currentSmoker' not found in df.")


,currentSmoker,chd_rate_pct
0,No,14.732143
1,Yes,15.979479


In [17]:
# 7.3) Tỷ lệ TenYearCHD theo diabetes (%)

c = "diabetes"
if c in df.columns:
    tmp = df[[c, "TenYearCHD"]].copy()

    # Clean target để tránh lỗi + đảm bảo đúng kiểu 0/1
    tmp["TenYearCHD"] = pd.to_numeric(tmp["TenYearCHD"], errors="coerce").replace([np.inf, -np.inf], np.nan)
    tmp = tmp.dropna(subset=[c, "TenYearCHD"])
    tmp["TenYearCHD"] = tmp["TenYearCHD"].astype(int)

    # mean(target) * 100 = % CHD trong từng nhóm diabetes
    rate = (tmp.groupby(c, dropna=False)["TenYearCHD"].mean() * 100).reset_index()
    rate = rate.rename(columns={"TenYearCHD": "chd_rate_pct"})
    display(rate)

    if HAS_PLOTLY:
        fig = px.bar(rate, x=c, y="chd_rate_pct", text="chd_rate_pct",
                     title=f"TenYearCHD rate by {c} (%)")
        fig.update_traces(texttemplate="%{text:.2f}%", textposition="outside")
        fig.update_layout(yaxis_title="CHD rate (%)", xaxis_title=c)
        fig.show()
    else:
        fig, ax = plt.subplots()
        ax.bar(rate[c].astype(str), rate["chd_rate_pct"])
        ax.set_title(f"TenYearCHD rate by {c} (%)")
        ax.set_ylabel("CHD rate (%)"); ax.set_xlabel(c)
        plt.show()
else:
    print("Column 'diabetes' not found in df.")


,diabetes,chd_rate_pct
0,No,14.767683
1,Yes,37.086093


**(8) Violinplot theo TenYearCHD — vẽ gì và dùng để làm gì?**

Cell này vẽ **violin plot** để so sánh phân phối của một biến số (mặc định ưu tiên `sysBP`, nếu không có thì dùng `age`) giữa 2 nhóm:

- `TenYearCHD = 0` (không mắc CHD)
- `TenYearCHD = 1` (mắc CHD)

**Violin plot thể hiện gì?**
- Phần “violin” (hình giống cây đàn) cho biết **mật độ phân phối**: chỗ nào phình to nghĩa là nhiều điểm dữ liệu tập trung ở đó.
- Nếu `box=True` (Plotly) / `inner="box"` (Seaborn), bên trong violin có thêm **boxplot**:
  - đường giữa: **median**
  - hộp: **IQR (Q1–Q3)**
  - râu: phạm vi dữ liệu (tuỳ cách tính)
- `points="outliers"` hiển thị các **điểm outlier**.

**Bạn đọc biểu đồ như nào?**
- Nếu nhóm `TenYearCHD=1` có violin/box “dịch lên cao” so với nhóm `0`, có thể biến đó (ví dụ `sysBP`) cao hơn ở nhóm mắc CHD.
- Nhìn độ rộng violin để biết nhóm nào phân tán rộng/hẹp, có nhiều outlier hay không.

Cell có fallback:
- Nếu có Plotly → dùng `px.violin` (đẹp và trực quan).
- Nếu không có Plotly mà có seaborn → `sns.violinplot`.
- Nếu thiếu cả seaborn → fallback sang boxplot (matplotlib).


In [18]:
# (8) Violinplot theo target: so sánh phân phối của 1 feature numeric giữa TenYearCHD=0 và TenYearCHD=1

# Chọn feature để vẽ: ưu tiên sysBP, nếu không có thì dùng age
feat = "sysBP" if "sysBP" in df.columns else ("age" if "age" in df.columns else None)
assert feat is not None, "No suitable numeric feature found for violin/ECDF."

# Lấy dữ liệu cần thiết và loại bỏ missing để plot không lỗi
tmp = df[[feat, "TenYearCHD"]].copy()

# Làm sạch target để tránh lỗi astype(int) do NaN/inf/string lạ
tmp["TenYearCHD"] = pd.to_numeric(tmp["TenYearCHD"], errors="coerce").replace([np.inf, -np.inf], np.nan)
tmp = tmp.dropna(subset=[feat, "TenYearCHD"])

# Ép target về int (0/1) rồi chuyển sang str để coi như category khi plot
tmp["TenYearCHD"] = tmp["TenYearCHD"].astype(int).astype(str)

if HAS_PLOTLY:
    # Plotly violin + box: violin cho hình dạng phân phối, box cho median/IQR, points hiển thị outliers
    fig = px.violin(
        tmp,
        x="TenYearCHD",
        y=feat,
        box=True,
        points="outliers",
        title=f"{feat} by TenYearCHD (violin + box)"
    )
    fig.update_layout(xaxis_title="TenYearCHD", yaxis_title=feat)
    fig.show()

else:
    # Nếu không có Plotly: ưu tiên seaborn violinplot; nếu không có seaborn thì fallback boxplot
    if HAS_SEABORN:
        import seaborn as sns
        fig, ax = plt.subplots()
        sns.violinplot(data=tmp, x="TenYearCHD", y=feat, inner="box", ax=ax)
        ax.set_title(f"{feat} by TenYearCHD (violin)")
        plt.show()
    else:
        fig, ax = plt.subplots()
        ax.boxplot(
            [tmp.loc[tmp["TenYearCHD"] == "0", feat],
             tmp.loc[tmp["TenYearCHD"] == "1", feat]],
            labels=["0", "1"]
        )
        ax.set_title(f"{feat} by TenYearCHD (box fallback)")
        plt.show()


**9) Train/Test split (stratify) + làm sạch target**

Cell này dùng để chia dữ liệu thành tập train/test theo tỷ lệ 80/20, đồng thời **giữ nguyên tỷ lệ class (0/1)** nhờ `stratify=y`.

- Làm sạch target `TenYearCHD`: ép về numeric và loại bỏ `NaN`, `inf`, `-inf` để tránh lỗi khi ép kiểu `int`.
- Tạo `X` và `y` chỉ từ các dòng có target hợp lệ.
- Thực hiện `train_test_split(..., stratify=y)` để train/test có tỷ lệ dương tính tương tự nhau.
- In ra shape và “pos rate” của train/test để kiểm tra stratify hoạt động đúng.


In [19]:
# 9) Train/Test split with stratify (giữ tỷ lệ class 0/1) + làm sạch target để tránh lỗi NaN/inf
TARGET = "TenYearCHD"

# Làm sạch y: ép về numeric, chuyển inf/-inf -> NaN, rồi lấy index các dòng có target hợp lệ
y_raw = pd.to_numeric(df[TARGET], errors="coerce").replace([np.inf, -np.inf], np.nan)
valid_idx = y_raw.dropna().index

# Tạo X, y chỉ trên các dòng valid (tránh leak/lỗi do target NaN/inf)
X = df.loc[valid_idx].drop(columns=[TARGET])
y = y_raw.loc[valid_idx].astype(int)

# (Tuỳ chọn) Lọc y chỉ còn 0/1 nếu dữ liệu có giá trị khác
# mask = y.isin([0, 1])
# X, y = X.loc[mask], y.loc[mask]

# Chia train/test có stratify để giữ tỷ lệ dương/âm tương tự nhau ở cả 2 tập
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

# In kích thước và tỷ lệ dương tính (pos rate) để kiểm tra stratify hoạt động đúng
print("Train:", X_train.shape, "Test:", X_test.shape)
print("Train pos rate:", round(float(y_train.mean()), 4), "Test pos rate:", round(float(y_test.mean()), 4))


Train: (8511, 15) Test: (2128, 15)
Train pos rate: 0.1534 Test pos rate: 0.1537


**8) Preprocess với ColumnTransformer (numeric + categorical)**

Cell này xây dựng pipeline tiền xử lý dữ liệu bằng `ColumnTransformer` để dùng chung cho mọi model.

- Tách danh sách cột thành 2 nhóm: **numeric** và **categorical** (chỉ lấy những cột thật sự tồn tại trong `X`).
- Với numeric: điền khuyết thiếu bằng **median** rồi **standardize** bằng `StandardScaler`.
- Với categorical: điền khuyết thiếu bằng **most_frequent** rồi **One-Hot Encode**.
- Hàm `make_ohe()` giúp tương thích nhiều phiên bản scikit-learn (`sparse_output` vs `sparse`).
- Kết quả `preprocess` sẽ được gắn vào pipeline model để đảm bảo preprocessing chạy đúng trong CV/train/test.


In [20]:
# 8) Xây dựng preprocess bằng ColumnTransformer: xử lý numeric/categorical (impute + scale/ohe) để đưa vào pipeline model

def make_ohe():
    # Tạo OneHotEncoder tương thích nhiều version sklearn:
    # - sklearn mới dùng sparse_output
    # - sklearn cũ dùng sparse
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)

# Chọn các cột categorical nếu chúng tồn tại trong X (tránh lỗi nếu thiếu cột)
categorical_cols = [
    c for c in ["gender","education","currentSmoker","BPMeds","prevalentStroke","prevalentHyp","diabetes"]
    if c in X.columns
]

# Chọn các cột numeric nếu chúng tồn tại trong X
numeric_cols = [
    c for c in ["age","cigsPerDay","totChol","sysBP","diaBP","BMI","heartRate","glucose"]
    if c in X.columns
]

# Pipeline cho numeric:
# - Điền missing bằng median (ít nhạy với outlier)
# - Scale về cùng thang đo (quan trọng với LR/SVM/KNN...)
numeric_tf = SkPipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Pipeline cho categorical:
# - Điền missing bằng mode (giá trị xuất hiện nhiều nhất)
# - One-hot encode, ignore category mới khi gặp ở test
categorical_tf = SkPipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", make_ohe())
])

# ColumnTransformer: áp pipeline numeric cho numeric_cols, categorical cho categorical_cols
preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_tf, numeric_cols),
        ("cat", categorical_tf, categorical_cols),
    ]
)

# In ra để kiểm tra danh sách cột đang được xử lý
print("Numeric:", numeric_cols)
print("Categorical:", categorical_cols)


Numeric: ['age', 'cigsPerDay', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']
Categorical: ['gender', 'education', 'currentSmoker', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes']


**9) Định nghĩa danh sách mô hình (12 models)**

Cell này import và chuẩn bị các thuật toán classification sẽ được benchmark trong các bước CV baseline và tuning.

- Gồm nhiều nhóm mô hình: tuyến tính (Logistic Regression), xác suất (Naive Bayes), dựa trên khoảng cách (KNN), biên quyết định (SVC), cây quyết định và các mô hình ensemble/boosting (RF, ExtraTrees, AdaBoost, GradientBoosting, HistGradientBoosting).
- Thêm 2 mô hình boosting phổ biến ngoài sklearn: **XGBoost** (`XGBClassifier`) và **LightGBM** (`LGBMClassifier`).
- `MODEL_NAMES` là danh sách nhãn ngắn gọn để dùng khi lưu kết quả và tạo bảng so sánh.


In [21]:
# 9) Import & chuẩn bị các model classification để benchmark (baseline CV) và tuning sau này

# Nhóm mô hình cơ bản
from sklearn.linear_model import LogisticRegression          # Logistic Regression (tuyến tính + prob)
from sklearn.naive_bayes import GaussianNB                   # Naive Bayes (giả định phân phối Gaussian)
from sklearn.neighbors import KNeighborsClassifier           # KNN (dựa trên khoảng cách)
from sklearn.svm import SVC                                  # SVM (biên quyết định; có thể bật probability)
from sklearn.tree import DecisionTreeClassifier              # Cây quyết định đơn

# Nhóm ensemble / boosting trong sklearn
from sklearn.ensemble import (
    RandomForestClassifier,                                  # Bagging cây (RF)
    ExtraTreesClassifier,                                    # Randomized trees (ET)
    AdaBoostClassifier,                                      # Boosting (AdaBoost)
    GradientBoostingClassifier,                              # Gradient Boosting truyền thống
    HistGradientBoostingClassifier                           # Gradient Boosting nhanh (hist-based)
)

# Boosting library ngoài sklearn (cần cài đặt)
from xgboost import XGBClassifier                            # XGBoost
from lightgbm import LGBMClassifier                          # LightGBM

# Tên viết tắt để hiển thị/bảng kết quả (12 models)
MODEL_NAMES = ['LR','GNB','KNN','SVC','DT','RF','ET','ADA','GB','HGB','XGB','LGBM']


**Helper: khởi tạo danh sách baseline models (có điều kiện XGB/LGBM)**

Cell này định nghĩa hàm `get_base_models()` để tạo một dict các mô hình baseline theo key (LR, RF, …).
Mục tiêu là chuẩn hoá việc “khởi tạo model ban đầu” trước khi chạy CV baseline.

- Trả về `models` dạng `{model_name: estimator}` để dễ loop qua nhiều model.
- Một số model được set `class_weight="balanced"` (hoặc `balanced_subsample`) để hỗ trợ dữ liệu mất cân bằng.
- `random_state=SEED` giúp tái lập kết quả.
- XGBoost và LightGBM chỉ được thêm vào nếu môi trường có cài đặt (`HAS_XGB`, `HAS_LGBM`).


In [22]:
# Helper: tạo dict các baseline models (init sẵn hyperparams cơ bản + xử lý imbalanced bằng class_weight khi phù hợp)

def get_base_models():
    models = {}

    # Logistic Regression: tăng max_iter để chắc chắn hội tụ; class_weight balanced để cân bằng lớp
    models["LR"]  = LogisticRegression(max_iter=2000, class_weight="balanced", random_state=SEED)

    # GaussianNB: baseline đơn giản, không cần nhiều hyperparams
    models["GNB"] = GaussianNB()

    # KNN: để mặc định (sẽ tuning sau nếu cần)
    models["KNN"] = KNeighborsClassifier()

    # SVC (RBF): class_weight balanced cho imbalanced; (chú ý: muốn predict_proba cần probability=True)
    models["SVC"] = SVC(C=1.0, gamma="scale", kernel="rbf", class_weight="balanced")

    # Decision Tree: class_weight balanced để giảm bias về majority class
    models["DT"]  = DecisionTreeClassifier(class_weight="balanced", random_state=SEED)

    # Random Forest: nhiều cây hơn để ổn định; balanced_subsample cân bằng theo từng bootstrap sample
    models["RF"]  = RandomForestClassifier(
        n_estimators=300,
        class_weight="balanced_subsample",
        random_state=SEED,
        n_jobs=-1
    )

    # Extra Trees: tương tự RF nhưng random mạnh hơn; thường baseline tốt, chạy nhanh
    models["ET"]  = ExtraTreesClassifier(
        n_estimators=500,
        class_weight="balanced_subsample",
        random_state=SEED,
        n_jobs=-1
    )

    # AdaBoost: baseline boosting (không có class_weight trực tiếp như LR/Tree)
    models["ADA"] = AdaBoostClassifier(random_state=SEED)

    # Gradient Boosting (classic)
    models["GB"]  = GradientBoostingClassifier(random_state=SEED)

    # HistGradientBoosting: boosting dạng histogram, thường nhanh hơn trên data lớn
    models["HGB"] = HistGradientBoostingClassifier(random_state=SEED)

    # Chỉ thêm XGBoost nếu đã cài (tránh lỗi import/runtime)
    if HAS_XGB:
        models["XGB"] = XGBClassifier(
            n_estimators=500,
            learning_rate=0.05,
            max_depth=4,
            subsample=0.8,
            colsample_bytree=0.8,
            eval_metric="logloss",
            random_state=SEED,
            n_jobs=-1
        )

    # Chỉ thêm LightGBM nếu đã cài
    if HAS_LGBM:
        models["LGBM"] = LGBMClassifier(
            n_estimators=800,
            learning_rate=0.03,
            num_leaves=31,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=SEED,
            n_jobs=-1
        )

    # Trả về dict để loop chạy CV baseline/tuning dễ dàng
    return models


**Pipeline + Cross-Validation evaluation (nhiều metrics)**
Cell này tạo **pipeline huấn luyện chuẩn** (preprocess → optional oversampling → model) và định nghĩa hàm chạy **cross-validation** để lấy điểm trung bình theo nhiều metric.

- `PIPE_CLS`: chọn `imblearn.Pipeline` nếu có imbalanced-learn (để dùng sampler trong pipeline), nếu không thì dùng `sklearn.Pipeline`.
- `build_pipeline(clf, use_sampler=True)`: ghép các bước:
  1) `preprocess` (ColumnTransformer đã định nghĩa trước)
  2) `RandomOverSampler` (nếu có imblearn và bật `use_sampler`) để cân bằng class trong train-fold
  3) `model` (classifier truyền vào)
- `scoring`: map tên metric → scoring string của sklearn để `cross_validate` chấm nhiều metric cùng lúc.
- `cv_evaluate(...)`: chạy `cross_validate` trên tập train với CV (thường StratifiedKFold), lấy **mean** score của từng metric và trả về dict kết quả. Nếu lỗi (ví dụ model không hỗ trợ), hàm trả về message lỗi để log/debug.


In [23]:
# Tạo pipeline (preprocess + optional sampler + model) và hàm chạy CV để đánh giá nhiều metrics

# Chọn Pipeline class:
# - Nếu có imbalanced-learn: dùng ImbPipeline để nhúng sampler vào pipeline đúng chuẩn (tránh data leakage)
# - Nếu không: dùng sklearn Pipeline
PIPE_CLS = ImbPipeline if HAS_IMBLEARN else SkPipeline

def build_pipeline(clf, use_sampler=True):
    # Bước 1: luôn luôn preprocess (impute/scale/ohe) trước khi train model
    steps = [("preprocess", preprocess)]

    # Bước 2 (tuỳ chọn): oversample để cân bằng class (chỉ hợp lệ khi có imblearn)
    if HAS_IMBLEARN and use_sampler:
        steps.append(("sampler", RandomOverSampler(random_state=SEED)))
    elif use_sampler and (not HAS_IMBLEARN):
        # Nếu thiếu imblearn thì không resampling được trong pipeline -> cảnh báo để bạn biết
        print("WARNING: imbalanced-learn missing -> no resampling; using class_weight where possible.")

    # Bước 3: gắn classifier cuối pipeline
    steps.append(("model", clf))

    # Trả về pipeline hoàn chỉnh để fit/predict/CV
    return PIPE_CLS(steps)

# Bộ metric dùng trong cross_validate (đánh giá nhiều tiêu chí cùng lúc)
scoring = {
    "pr_auc": "average_precision",      # PR-AUC (tốt cho imbalanced)
    "roc_auc": "roc_auc",               # ROC-AUC
    "recall": "recall",                 # Recall
    "precision": "precision",           # Precision
    "f1": "f1",                         # F1-score
    "bal_acc": "balanced_accuracy"      # Balanced Accuracy
}

def cv_evaluate(model_name, pipe, Xtr, ytr, cv):
    # Dict kết quả tối thiểu luôn có tên model để dễ tổng hợp bảng
    out = {"model": model_name}
    try:
        # Chạy cross-validation:
        # - fit pipeline trên train folds
        # - evaluate trên valid fold
        # - trả về scores cho từng metric ở từng fold
        res = cross_validate(
            pipe, Xtr, ytr,
            cv=cv,
            scoring=scoring,
            n_jobs=-1,
            error_score="raise"   # nếu lỗi thì raise để catch ở except và log error rõ ràng
        )

        # Lấy trung bình score qua các folds cho mỗi metric
        for k in scoring.keys():
            out[k] = float(np.mean(res[f"test_{k}"]))

        # Trả về kết quả + None (không lỗi)
        return out, None

    except Exception as e:
        # Nếu lỗi (vd model không tương thích/scoring fail), trả về dict + message lỗi để debug
        return out, str(e)


## Huấn luyện & tuning theo từng model (tách cell để quan sát quy trình)

- **Baseline**: mỗi model được huấn luyện và đánh giá bằng **OOF predictions trên TRAIN** (StratifiedKFold) để tránh leakage.
- **Tuning**: chỉ thực hiện **sau khi baseline của tất cả model hoàn tất**.
- Mỗi bước đều có:
  - **Metrics** cho classification
  - **Confusion matrix** (OOF TRAIN)
  - **ROC curve** (OOF TRAIN)

> Lưu ý y tế: phần đánh giá TEST vẫn chỉ dùng 1 lần ở cuối (không dùng test để chọn threshold).


**Setup CV + imports phục vụ tuning/đánh giá + các biến lưu kết quả**

Cell này chuẩn bị các thành phần cần thiết cho toàn bộ giai đoạn **CV baseline**, **tuning (RandomizedSearchCV)** và **đánh giá cuối**.

- Import các công cụ CV/tuning (`StratifiedKFold`, `RandomizedSearchCV`, `cross_val_predict`).
- Import các metric và hàm vẽ curve (ROC/PR), confusion matrix, classification report, và Brier score (đánh giá calibration xác suất).
- Import các phân phối xác suất từ `scipy.stats` để khai báo không gian tìm kiếm cho `RandomizedSearchCV` (loguniform/randint/uniform).
- Tạo `cv5`: Stratified 5-Fold có shuffle, đảm bảo mỗi fold giữ tỷ lệ lớp 0/1 tương tự nhau và tái lập kết quả nhờ `SEED`.
- Khởi tạo các “container” để lưu kết quả baseline, kết quả sau tuning và best hyperparameters của từng model.


In [24]:
# Setup: import công cụ CV/tuning + metrics + distributions, và tạo biến lưu kết quả

# Import công cụ cho cross-validation, tuning và dự đoán OOF (out-of-fold)
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_val_predict

# Import các metric/hàm đánh giá:
# - ROC-AUC, PR-AUC + ROC/PR curves
# - Confusion matrix, classification report
# - Brier score để đánh giá calibration (xác suất dự đoán có "đúng xác suất" không)
from sklearn.metrics import (
    roc_auc_score, average_precision_score, roc_curve, precision_recall_curve,
    confusion_matrix, classification_report, brier_score_loss
)

# Import phân phối dùng cho RandomizedSearchCV (sample hyperparams theo phân phối)
from scipy.stats import loguniform, randint, uniform

# Tạo CV 5-fold dạng stratified (giữ tỷ lệ lớp) + shuffle để chia ngẫu nhiên, seed để tái lập
cv5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# Containers để lưu kết quả qua từng model
baseline_results = []   # lưu điểm baseline CV của từng model (mean metrics)
tuned_results = []      # lưu điểm sau tuning CV của từng model
best_params = {}        # dict lưu best hyperparameters theo model_name


**Helper: OOF scores + chọn threshold ưu tiên Recall**

Cell này định nghĩa 2 hàm hỗ trợ cho bước **tối ưu ngưỡng phân loại** dựa trên dự đoán **OOF (Out-Of-Fold)**.

- `get_oof_scores(...)`: tạo OOF scores cho toàn bộ dữ liệu bằng `cross_val_predict`. Hàm ưu tiên lấy **xác suất lớp 1** từ `predict_proba`. Nếu model không có `predict_proba` thì dùng `decision_function` và chuẩn hoá về [0,1]. Nếu vẫn không có thì fallback sang `predict` (0/1).
- `pick_threshold_by_recall(...)`: dựa trên **precision-recall curve** để chọn threshold sao cho **Recall cao nhất**, nhưng vẫn cố gắng đảm bảo **Precision >= min_precision** (mặc định 0.15). Nếu không có threshold nào đạt min_precision thì chọn threshold cho Recall cao nhất bất chấp precision.

Mục tiêu: tăng khả năng “bắt bệnh” (Recall) khi dữ liệu mất cân bằng, đồng thời tránh precision quá thấp.


In [25]:
# Helper: lấy OOF scores (out-of-fold) + chọn threshold ưu tiên Recall (có ràng buộc precision tối thiểu)

def get_oof_scores(estimator, X, y, cv):
    # Trả về OOF scores trong [0,1].
    # Ưu tiên predict_proba (xác suất), nếu không có thì dùng decision_function (chuẩn hoá về [0,1]),
    # cuối cùng fallback predict (0/1) nếu model không hỗ trợ cả hai.
    try:
        # Cố gắng lấy xác suất lớp dương (class=1) theo kiểu OOF qua cross-validation
        proba = cross_val_predict(
            estimator, X, y,
            cv=cv, method="predict_proba",
            n_jobs=-1
        )[:, 1]
        return proba.astype(float)

    except Exception:
        try:
            # Nếu không có predict_proba, dùng decision_function (điểm quyết định)
            scores = cross_val_predict(
                estimator, X, y,
                cv=cv, method="decision_function",
                n_jobs=-1
            ).astype(float)

            # Chuẩn hoá min-max về [0,1] để dùng cho PR curve/threshold
            smin, smax = np.min(scores), np.max(scores)

            # Nếu scores gần như hằng -> tránh chia 0, trả về 0.5
            if smax - smin < 1e-12:
                return np.full_like(scores, 0.5, dtype=float)

            return (scores - smin) / (smax - smin)

        except Exception:
            # Fallback cuối: chỉ lấy nhãn dự đoán OOF (0/1) nếu không có proba/decision_function
            preds = cross_val_predict(
                estimator, X, y,
                cv=cv, method="predict",
                n_jobs=-1
            )
            return preds.astype(float)


def pick_threshold_by_recall(y_true, scores, min_precision=0.15):
    # Chọn threshold dựa trên Precision-Recall curve:
    # - Ưu tiên Recall cao nhất trong nhóm threshold có Precision >= min_precision
    # - Nếu không có threshold nào đạt min_precision, chọn threshold có Recall cao nhất (bỏ qua precision)
    prec, rec, thr = precision_recall_curve(y_true, scores)

    # Trường hợp hiếm: không có threshold (ví dụ scores hằng) -> dùng default 0.5
    if len(thr) == 0:
        return 0.5

    # prec/rec dài hơn thr 1 phần tử -> cắt bỏ phần tử cuối để khớp với thr
    prec2, rec2 = prec[:-1], rec[:-1]

    # Lọc các threshold thỏa precision tối thiểu
    mask = prec2 >= min_precision

    if mask.any():
        # Trong nhóm đạt precision tối thiểu, lấy threshold cho recall lớn nhất
        idx = np.argmax(rec2[mask])
        return float(thr[mask][idx])

    # Nếu không có threshold nào đạt precision tối thiểu, lấy threshold cho recall lớn nhất
    return float(thr[np.argmax(rec2)])

**Helper: tính metrics theo threshold + vẽ Confusion Matrix/ROC (Plotly, fallback Matplotlib)**

Cell này định nghĩa các hàm hỗ trợ để **đánh giá mô hình theo xác suất dự đoán (scores)** và **một ngưỡng phân loại (threshold)**, đồng thời vẽ các biểu đồ quan trọng.

- `compute_metrics(y_true, scores, thr)`:  
  Chuyển `scores` → nhãn dự đoán bằng ngưỡng `thr`, sau đó tính confusion matrix và các metric chính:
  ROC-AUC, PR-AUC, Recall, Precision, F1, Specificity, Balanced Accuracy và Brier score (đánh giá calibration của xác suất).

- `plot_cm(cm, ...)`:  
  Vẽ confusion matrix bằng Plotly (heatmap + số trong ô). Nếu không có Plotly thì fallback sang Matplotlib.

- `plot_roc(y_true, scores, ...)`:  
  Tính ROC curve (FPR/TPR) và vẽ bằng Plotly. Nếu không có Plotly thì fallback Matplotlib.

Mục tiêu: sau khi chọn threshold, bạn có thể xem nhanh **độ bắt bệnh (recall)**, **độ chính xác (precision)**, và hình ảnh trực quan về sai/đúng qua confusion matrix và ROC.


In [26]:
# Helper: tính metrics theo threshold + vẽ Confusion Matrix và ROC (ưu tiên Plotly, thiếu thì fallback Matplotlib)

def compute_metrics(y_true, scores, thr):
    # Chuyển scores (xác suất/điểm) thành nhãn dự đoán theo threshold
    y_pred = (scores >= thr).astype(int)

    # Tính confusion matrix và tách TN/FP/FN/TP để tự tính thêm các metric
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    # Tính các metric theo định nghĩa (có kiểm tra chia 0)
    recall = tp / (tp + fn) if (tp + fn) else 0.0                 # Sensitivity / TPR
    precision = tp / (tp + fp) if (tp + fp) else 0.0              # PPV
    specificity = tn / (tn + fp) if (tn + fp) else 0.0            # TNR
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0
    bal_acc = 0.5 * (recall + specificity)                        # Balanced Accuracy

    # Trả về dict metric (AUC tính trực tiếp từ scores) + confusion matrix
    return {
        "roc_auc": roc_auc_score(y_true, scores),                 # ROC-AUC
        "pr_auc": average_precision_score(y_true, scores),        # PR-AUC (Average Precision)
        "recall": recall,
        "precision": precision,
        "f1": f1,
        "specificity": specificity,
        "bal_acc": bal_acc,
        "brier": brier_score_loss(y_true, scores)                 # Brier score (calibration)
    }, cm


def plot_cm(cm, title="Confusion Matrix"):
    # Vẽ confusion matrix: ưu tiên Plotly heatmap, nếu không có thì dùng Matplotlib
    if HAS_PLOTLY:
        fig = go.Figure(
            data=go.Heatmap(
                z=cm,
                x=["Pred 0", "Pred 1"],
                y=["True 0", "True 1"]
            )
        )
        fig.update_layout(title=title, xaxis_title="Predicted", yaxis_title="True")

        # Thêm số lên từng ô để dễ đọc
        for i in range(2):
            for j in range(2):
                fig.add_annotation(x=j, y=i, text=str(cm[i, j]), showarrow=False)

        fig.show()
    else:
        fig, ax = plt.subplots()
        ax.imshow(cm)

        # In số lên từng ô
        for (i, j), v in np.ndenumerate(cm):
            ax.text(j, i, str(v), ha="center", va="center")

        ax.set_title(title)
        ax.set_xlabel("Predicted")
        ax.set_ylabel("True")
        plt.show()


def plot_roc(y_true, scores, title="ROC"):
    # Tính ROC curve từ y_true và scores
    fpr, tpr, _ = roc_curve(y_true, scores)

    # Vẽ ROC: ưu tiên Plotly, thiếu thì dùng Matplotlib
    if HAS_PLOTLY:
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=fpr, y=tpr, mode="lines", name="ROC"))
        fig.add_trace(go.Scatter(
            x=[0, 1], y=[0, 1],
            mode="lines", name="Chance",
            line=dict(dash="dash")
        ))
        fig.update_layout(
            title=title,
            xaxis_title="False Positive Rate",
            yaxis_title="True Positive Rate"
        )
        fig.show()
    else:
        fig, ax = plt.subplots()
        ax.plot(fpr, tpr, label="ROC")
        ax.plot([0, 1], [0, 1], linestyle="--", label="Chance")
        ax.set_title(title)
        ax.set_xlabel("FPR")
        ax.set_ylabel("TPR")
        ax.legend()
        plt.show()

## Baseline: huấn luyện & đánh giá từng model (OOF TRAIN)

### LR — Logistic Regression (Baseline, OOF TRAIN)

In [27]:
# [LR - Cell 1] Build baseline LR pipeline (không oversample để tránh double-imbalance handling)

from sklearn.linear_model import LogisticRegression

# LR baseline: class_weight balanced để xử lý imbalanced, max_iter tăng để hội tụ tốt hơn
lr_base = LogisticRegression(
    max_iter=2000,
    solver="liblinear",
    class_weight="balanced",
    random_state=SEED
)

# IMPORTANT: tắt sampler cho LR khi đã dùng class_weight (tránh đẩy proba lệch mạnh về class 1)
lr_pipe_base = build_pipeline(lr_base, use_sampler=False)


In [28]:
# [LR - Cell 2] OOF scores + chọn threshold (tối ưu F1) + lưu baseline_results

# Lấy OOF scores (xác suất class 1) trên TRAIN bằng CV
lr_oof = get_oof_scores(lr_pipe_base, X_train, y_train, cv5)

# Chọn threshold theo F1 (ổn định hơn "ưu tiên recall" và tránh predict-all-positive)
prec, rec, thr = precision_recall_curve(y_train, lr_oof)
if len(thr) == 0:
    lr_thr = 0.37
else:
    f1s = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
    lr_thr = float(thr[np.argmax(f1s)])

# Tính metrics theo threshold vừa chọn (trên OOF TRAIN)
lr_m, lr_cm = compute_metrics(y_train, lr_oof, lr_thr)

# Lưu vào bảng baseline
baseline_results.append({"model": "LR", "phase": "baseline", "status": "OK", "thr": lr_thr, **lr_m})

print("Chosen threshold (F1) =", round(lr_thr, 4))
print("Predicted positive rate =", round(float((lr_oof >= lr_thr).mean()), 4))
display(pd.DataFrame([lr_m]).round(4))


Chosen threshold (F1) = 0.5323
Predicted positive rate = 0.3335


,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier
0,0.7216,0.3419,0.6133,0.2822,0.3866,0.7173,0.6653,0.2119


In [29]:
# [LR - Cell 3] ROC-AUC + Classification report + Confusion Matrix + ROC (OOF TRAIN)

lr_pred_oof = (lr_oof >= lr_thr).astype(int)

# ROC-AUC (OOF TRAIN) — metric chính
lr_roc_auc = roc_auc_score(y_train, lr_oof)
print("ROC-AUC (OOF TRAIN) =", round(float(lr_roc_auc), 4))

# Classification report (theo threshold đã chọn)
print(classification_report(y_train, lr_pred_oof, digits=3))

# Confusion Matrix (Plotly cho đẹp)
if HAS_PLOTLY:
    cm = confusion_matrix(y_train, lr_pred_oof)
    fig = go.Figure(
        data=go.Heatmap(
            z=cm,
            x=["Pred 0", "Pred 1"],
            y=["True 0", "True 1"],
            colorscale="Blues",
            zmin=0,
            zmax=int(cm.max())
        )
    )
    fig.update_layout(
        title="LR Baseline - Confusion Matrix (OOF TRAIN)",
        xaxis_title="Predicted",
        yaxis_title="True"
    )
    for i in range(2):
        for j in range(2):
            fig.add_annotation(
                x=j, y=i, text=str(cm[i, j]),
                showarrow=False, font=dict(size=16)
            )
    fig.show()
else:
    plot_cm(confusion_matrix(y_train, lr_pred_oof),
            title="LR Baseline - Confusion Matrix (OOF TRAIN)")

# ROC curve
plot_roc(y_train, lr_oof, title="LR Baseline - ROC (OOF TRAIN)")


ROC-AUC (OOF TRAIN) = 0.7216
              precision    recall  f1-score   support

           0      0.911     0.717     0.803      7205
           1      0.282     0.613     0.387      1306

    accuracy                          0.701      8511
   macro avg      0.597     0.665     0.595      8511
weighted avg      0.815     0.701     0.739      8511



### GNB — Gaussian Naive Bayes (Baseline, OOF TRAIN)

In [30]:
# [GNB - Cell 1] Build baseline GaussianNB pipeline
# Lưu ý: GaussianNB KHÔNG nên dùng oversampling (RandomOverSampler)
# vì NB giả định phân phối xác suất -> oversample sẽ làm phân phối bị méo

from sklearn.naive_bayes import GaussianNB

# GaussianNB baseline (không có class_weight)
gnb_base = GaussianNB()

# TẮT sampler cho GNB
gnb_pipe_base = build_pipeline(gnb_base, use_sampler=False)


In [31]:
# [GNB - Cell 2] OOF scores + chọn threshold theo F1

# Lấy OOF scores (xác suất class 1) trên TRAIN bằng CV
gnb_oof = get_oof_scores(gnb_pipe_base, X_train, y_train, cv5)

# Chọn threshold theo F1 (ổn định hơn so với ưu tiên recall thuần)
prec, rec, thr = precision_recall_curve(y_train, gnb_oof)
if len(thr) == 0:
    gnb_thr = 0.5
else:
    f1s = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
    gnb_thr = float(thr[np.argmax(f1s)])

# Tính metrics theo threshold đã chọn (OOF TRAIN)
gnb_m, gnb_cm = compute_metrics(y_train, gnb_oof, gnb_thr)

# Lưu kết quả baseline
baseline_results.append({
    "model": "GNB",
    "phase": "baseline",
    "status": "OK",
    "thr": gnb_thr,
    **gnb_m
})

print("Chosen threshold (F1) =", round(gnb_thr, 4))
print("Predicted positive rate =", round(float((gnb_oof >= gnb_thr).mean()), 4))
display(pd.DataFrame([gnb_m]).round(4))


Chosen threshold (F1) = 0.0108
Predicted positive rate = 0.2658


,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier
0,0.7016,0.2891,0.4954,0.286,0.3627,0.7759,0.6356,0.1666


In [32]:
# [GNB - Cell 3] ROC-AUC + Classification report + Confusion Matrix + ROC (OOF TRAIN)

gnb_pred_oof = (gnb_oof >= gnb_thr).astype(int)

# ROC-AUC (OOF TRAIN) — metric chính
gnb_roc_auc = roc_auc_score(y_train, gnb_oof)
print("ROC-AUC (OOF TRAIN) =", round(float(gnb_roc_auc), 4))

# Classification report (theo threshold đã chọn)
print(classification_report(y_train, gnb_pred_oof, digits=3))

# Confusion Matrix (Plotly cho đẹp)
if HAS_PLOTLY:
    cm = gnb_cm
    fig = go.Figure(
        data=go.Heatmap(
            z=cm,
            x=["Pred 0", "Pred 1"],
            y=["True 0", "True 1"],
            colorscale="Blues",
            zmin=0,
            zmax=int(cm.max())
        )
    )
    fig.update_layout(
        title="GNB Baseline - Confusion Matrix (OOF TRAIN)",
        xaxis_title="Predicted",
        yaxis_title="True"
    )
    for i in range(2):
        for j in range(2):
            fig.add_annotation(
                x=j, y=i, text=str(cm[i, j]),
                showarrow=False, font=dict(size=16)
            )
    fig.show()
else:
    plot_cm(gnb_cm, title="GNB Baseline - Confusion Matrix (OOF TRAIN)")

# ROC curve
plot_roc(y_train, gnb_oof, title="GNB Baseline - ROC (OOF TRAIN)")


ROC-AUC (OOF TRAIN) = 0.7016
              precision    recall  f1-score   support

           0      0.895     0.776     0.831      7205
           1      0.286     0.495     0.363      1306

    accuracy                          0.733      8511
   macro avg      0.590     0.636     0.597      8511
weighted avg      0.801     0.733     0.759      8511



### KNN — K-Nearest Neighbors (Baseline, OOF TRAIN)

In [33]:
# [KNN - Cell 1] Build baseline KNN pipeline
# Lưu ý: KNN KHÔNG dùng oversampling (RandomOverSampler)
# vì KNN rất nhạy với khoảng cách

from sklearn.neighbors import KNeighborsClassifier

# KNN baseline (n_neighbors=15 như bạn chọn)
knn_base = KNeighborsClassifier(
    n_neighbors=15,
    weights="uniform"   # baseline, sẽ tuning sau
)

# TẮT sampler cho KNN
knn_pipe_base = build_pipeline(knn_base, use_sampler=False)


In [34]:
# [KNN - Cell 2] OOF scores + chọn threshold theo F1

# Lấy OOF scores (xác suất class 1) trên TRAIN bằng CV
knn_oof = get_oof_scores(knn_pipe_base, X_train, y_train, cv5)

# Chọn threshold tối ưu F1 (ổn định, tránh predict-all-positive)
prec, rec, thr = precision_recall_curve(y_train, knn_oof)
if len(thr) == 0:
    knn_thr = 0.5
else:
    f1s = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
    knn_thr = float(thr[np.argmax(f1s)])

# Tính metrics theo threshold đã chọn (OOF TRAIN)
knn_m, knn_cm = compute_metrics(y_train, knn_oof, knn_thr)

# Lưu kết quả baseline
baseline_results.append({
    "model": "KNN",
    "phase": "baseline",
    "status": "OK",
    "thr": knn_thr,
    **knn_m
})

print("Chosen threshold (F1) =", round(knn_thr, 4))
print("Predicted positive rate =", round(float((knn_oof >= knn_thr).mean()), 4))
display(pd.DataFrame([knn_m]).round(4))


Chosen threshold (F1) = 0.2667
Predicted positive rate = 0.2278


,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier
0,0.7716,0.3552,0.536,0.361,0.4314,0.828,0.682,0.1127


In [35]:
# [KNN - Cell 3] ROC-AUC + Classification report + Confusion Matrix + ROC (OOF TRAIN)

knn_pred_oof = (knn_oof >= knn_thr).astype(int)

# ROC-AUC (OOF TRAIN) — metric chính
knn_roc_auc = roc_auc_score(y_train, knn_oof)
print("ROC-AUC (OOF TRAIN) =", round(float(knn_roc_auc), 4))

# Classification report (theo threshold đã chọn)
print(classification_report(y_train, knn_pred_oof, digits=3))

# Confusion Matrix (Plotly cho đẹp)
if HAS_PLOTLY:
    cm = knn_cm
    fig = go.Figure(
        data=go.Heatmap(
            z=cm,
            x=["Pred 0", "Pred 1"],
            y=["True 0", "True 1"],
            colorscale="Blues",
            zmin=0,
            zmax=int(cm.max())
        )
    )
    fig.update_layout(
        title="KNN Baseline - Confusion Matrix (OOF TRAIN)",
        xaxis_title="Predicted",
        yaxis_title="True"
    )
    for i in range(2):
        for j in range(2):
            fig.add_annotation(
                x=j, y=i, text=str(cm[i, j]),
                showarrow=False, font=dict(size=16)
            )
    fig.show()
else:
    plot_cm(knn_cm, title="KNN Baseline - Confusion Matrix (OOF TRAIN)")

# ROC curve
plot_roc(y_train, knn_oof, title="KNN Baseline - ROC (OOF TRAIN)")


ROC-AUC (OOF TRAIN) = 0.7716
              precision    recall  f1-score   support

           0      0.908     0.828     0.866      7205
           1      0.361     0.536     0.431      1306

    accuracy                          0.783      8511
   macro avg      0.634     0.682     0.649      8511
weighted avg      0.824     0.783     0.799      8511



### SVC — Support Vector Classifier (RBF) (Baseline, OOF TRAIN)

In [36]:
# [SVC - Cell 1] Build baseline SVC pipeline
# Lưu ý:
# - KHÔNG oversample SVC
# - Dùng class_weight="balanced"
# - probability=True để lấy predict_proba cho ROC/threshold

from sklearn.svm import SVC

svc_base = SVC(
    C=1.0,
    kernel="rbf",
    gamma="scale",
    class_weight="balanced",
    probability=True,
    random_state=SEED
)

# TẮT sampler cho SVC
svc_pipe_base = build_pipeline(svc_base, use_sampler=False)


In [37]:
# [SVC - Cell 2] OOF scores + chọn threshold theo F1

# Lấy OOF scores (xác suất class 1) trên TRAIN bằng CV
svc_oof = get_oof_scores(svc_pipe_base, X_train, y_train, cv5)

# Chọn threshold tối ưu theo F1 (ổn định, tránh lệch recall cực đoan)
prec, rec, thr = precision_recall_curve(y_train, svc_oof)
if len(thr) == 0:
    svc_thr = 0.5
else:
    f1s = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
    svc_thr = float(thr[np.argmax(f1s)])

# Tính metrics theo threshold đã chọn (OOF TRAIN)
svc_m, svc_cm = compute_metrics(y_train, svc_oof, svc_thr)

# Lưu kết quả baseline
baseline_results.append({
    "model": "SVC",
    "phase": "baseline",
    "status": "OK",
    "thr": svc_thr,
    **svc_m
})

print("Chosen threshold (F1) =", round(svc_thr, 4))
print("Predicted positive rate =", round(float((svc_oof >= svc_thr).mean()), 4))
display(pd.DataFrame([svc_m]).round(4))


Chosen threshold (F1) = 0.26
Predicted positive rate = 0.2016


,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier
0,0.7713,0.372,0.526,0.4003,0.4547,0.8572,0.6916,0.1119


In [38]:
# [SVC - Cell 3] ROC-AUC + Classification report + Confusion Matrix + ROC (OOF TRAIN)

svc_pred_oof = (svc_oof >= svc_thr).astype(int)

# ROC-AUC (OOF TRAIN) — metric chính
svc_roc_auc = roc_auc_score(y_train, svc_oof)
print("ROC-AUC (OOF TRAIN) =", round(float(svc_roc_auc), 4))

# Classification report (theo threshold đã chọn)
print(classification_report(y_train, svc_pred_oof, digits=3))

# Confusion Matrix (Plotly cho đẹp)
if HAS_PLOTLY:
    cm = svc_cm
    fig = go.Figure(
        data=go.Heatmap(
            z=cm,
            x=["Pred 0", "Pred 1"],
            y=["True 0", "True 1"],
            colorscale="Blues",
            zmin=0,
            zmax=int(cm.max())
        )
    )
    fig.update_layout(
        title="SVC Baseline - Confusion Matrix (OOF TRAIN)",
        xaxis_title="Predicted",
        yaxis_title="True"
    )
    for i in range(2):
        for j in range(2):
            fig.add_annotation(
                x=j, y=i, text=str(cm[i, j]),
                showarrow=False, font=dict(size=16)
            )
    fig.show()
else:
    plot_cm(svc_cm, title="SVC Baseline - Confusion Matrix (OOF TRAIN)")

# ROC curve
plot_roc(y_train, svc_oof, title="SVC Baseline - ROC (OOF TRAIN)")


ROC-AUC (OOF TRAIN) = 0.7713
              precision    recall  f1-score   support

           0      0.909     0.857     0.882      7205
           1      0.400     0.526     0.455      1306

    accuracy                          0.806      8511
   macro avg      0.655     0.692     0.668      8511
weighted avg      0.831     0.806     0.817      8511



### DT — Decision Tree (Baseline, OOF TRAIN)

In [39]:
# [DT - Cell 1] Build baseline Decision Tree pipeline
# Lưu ý:
# - KHÔNG oversample DT (dễ overfit)
# - Dùng class_weight="balanced" để xử lý imbalance

from sklearn.tree import DecisionTreeClassifier

dt_base = DecisionTreeClassifier(
    class_weight="balanced",
    random_state=SEED
)

# TẮT sampler cho DT
dt_pipe_base = build_pipeline(dt_base, use_sampler=False)


In [40]:
# [DT - Cell 2] OOF scores + chọn threshold theo F1

# Lấy OOF scores (xác suất class 1) trên TRAIN bằng CV
dt_oof = get_oof_scores(dt_pipe_base, X_train, y_train, cv5)

# Chọn threshold tối ưu theo F1 (tránh lệch recall cực đoan)
prec, rec, thr = precision_recall_curve(y_train, dt_oof)
if len(thr) == 0:
    dt_thr = 0.5
else:
    f1s = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
    dt_thr = float(thr[np.argmax(f1s)])

# Tính metrics theo threshold đã chọn (OOF TRAIN)
dt_m, dt_cm = compute_metrics(y_train, dt_oof, dt_thr)

# Lưu kết quả baseline
baseline_results.append({
    "model": "DT",
    "phase": "baseline",
    "status": "OK",
    "thr": dt_thr,
    **dt_m
})

print("Chosen threshold (F1) =", round(dt_thr, 4))
print("Predicted positive rate =", round(float((dt_oof >= dt_thr).mean()), 4))
display(pd.DataFrame([dt_m]).round(4))


Chosen threshold (F1) = 1.0
Predicted positive rate = 0.1477


,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier
0,0.7262,0.3646,0.5306,0.5513,0.5408,0.9217,0.7262,0.1383


In [41]:
# [DT - Cell 3] ROC-AUC + Classification report + Confusion Matrix + ROC (OOF TRAIN)

dt_pred_oof = (dt_oof >= dt_thr).astype(int)

# In ROC-AUC (OOF TRAIN) — metric quan trọng cho bài này
dt_roc_auc = roc_auc_score(y_train, dt_oof)
print("ROC-AUC (OOF TRAIN) =", round(float(dt_roc_auc), 4))

# Classification report (theo threshold đã chọn)
print(classification_report(y_train, dt_pred_oof, digits=3))

# Confusion Matrix (Plotly cho đẹp)
if HAS_PLOTLY:
    cm = dt_cm
    fig = go.Figure(
        data=go.Heatmap(
            z=cm,
            x=["Pred 0", "Pred 1"],
            y=["True 0", "True 1"],
            colorscale="Blues",
            zmin=0,
            zmax=int(cm.max())
        )
    )
    fig.update_layout(
        title="DT Baseline - Confusion Matrix (OOF TRAIN)",
        xaxis_title="Predicted",
        yaxis_title="True"
    )
    for i in range(2):
        for j in range(2):
            fig.add_annotation(
                x=j, y=i, text=str(cm[i, j]),
                showarrow=False, font=dict(size=16)
            )
    fig.show()
else:
    plot_cm(dt_cm, title="DT Baseline - Confusion Matrix (OOF TRAIN)")

# ROC curve
plot_roc(y_train, dt_oof, title="DT Baseline - ROC (OOF TRAIN)")


ROC-AUC (OOF TRAIN) = 0.7262
              precision    recall  f1-score   support

           0      0.915     0.922     0.919      7205
           1      0.551     0.531     0.541      1306

    accuracy                          0.862      8511
   macro avg      0.733     0.726     0.730      8511
weighted avg      0.860     0.862     0.861      8511



### RF — Random Forest (Baseline, OOF TRAIN)

In [42]:
# [RF - Cell 1] Build baseline RandomForest pipeline
# Lưu ý:
# - KHÔNG oversample RF
# - class_weight="balanced" là đủ cho imbalance

from sklearn.ensemble import RandomForestClassifier

rf_base = RandomForestClassifier(
    n_estimators=400,
    class_weight="balanced",
    random_state=SEED,
    n_jobs=-1
)

# TẮT sampler cho RF
rf_pipe_base = build_pipeline(rf_base, use_sampler=False)


In [43]:
# [RF - Cell 2] OOF scores + chọn threshold theo F1

# Lấy OOF scores (xác suất class 1) trên TRAIN bằng CV
rf_oof = get_oof_scores(rf_pipe_base, X_train, y_train, cv5)

# Chọn threshold tối ưu theo F1 (ổn định, công bằng để so baseline)
prec, rec, thr = precision_recall_curve(y_train, rf_oof)
if len(thr) == 0:
    rf_thr = 0.5
else:
    f1s = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
    rf_thr = float(thr[np.argmax(f1s)])

# Tính metrics theo threshold đã chọn (OOF TRAIN)
rf_m, rf_cm = compute_metrics(y_train, rf_oof, rf_thr)

# Lưu kết quả baseline
baseline_results.append({
    "model": "RF",
    "phase": "baseline",
    "status": "OK",
    "thr": rf_thr,
    **rf_m
})

print("Chosen threshold (F1) =", round(rf_thr, 4))
print("Predicted positive rate =", round(float((rf_oof >= rf_thr).mean()), 4))
display(pd.DataFrame([rf_m]).round(4))


Chosen threshold (F1) = 0.28
Predicted positive rate = 0.1369


,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier
0,0.9381,0.8435,0.7175,0.8043,0.7584,0.9684,0.8429,0.0659


In [44]:
# [RF - Cell 3] ROC-AUC + Classification report + Confusion Matrix + ROC (OOF TRAIN)

rf_pred_oof = (rf_oof >= rf_thr).astype(int)

# ROC-AUC (OOF TRAIN) — metric chính cho so sánh model
rf_roc_auc = roc_auc_score(y_train, rf_oof)
print("ROC-AUC (OOF TRAIN) =", round(float(rf_roc_auc), 4))

# Classification report (theo threshold đã chọn)
print(classification_report(y_train, rf_pred_oof, digits=3))

# Confusion Matrix (Plotly cho đẹp)
if HAS_PLOTLY:
    cm = rf_cm
    fig = go.Figure(
        data=go.Heatmap(
            z=cm,
            x=["Pred 0", "Pred 1"],
            y=["True 0", "True 1"],
            colorscale="Blues",
            zmin=0,
            zmax=int(cm.max())
        )
    )
    fig.update_layout(
        title="RF Baseline - Confusion Matrix (OOF TRAIN)",
        xaxis_title="Predicted",
        yaxis_title="True"
    )
    for i in range(2):
        for j in range(2):
            fig.add_annotation(
                x=j, y=i, text=str(cm[i, j]),
                showarrow=False, font=dict(size=16)
            )
    fig.show()
else:
    plot_cm(rf_cm, title="RF Baseline - Confusion Matrix (OOF TRAIN)")

# ROC curve
plot_roc(y_train, rf_oof, title="RF Baseline - ROC (OOF TRAIN)")


ROC-AUC (OOF TRAIN) = 0.9381
              precision    recall  f1-score   support

           0      0.950     0.968     0.959      7205
           1      0.804     0.717     0.758      1306

    accuracy                          0.930      8511
   macro avg      0.877     0.843     0.859      8511
weighted avg      0.927     0.930     0.928      8511



### ET — Extra Trees (Baseline, OOF TRAIN)

In [45]:
# [ET - Cell 1] Build baseline ExtraTrees pipeline
# Lưu ý:
# - KHÔNG oversample ET
# - class_weight="balanced" là đủ

from sklearn.ensemble import ExtraTreesClassifier

et_base = ExtraTreesClassifier(
    n_estimators=400,
    class_weight="balanced",
    random_state=SEED,
    n_jobs=-1
)

# TẮT sampler cho ET
et_pipe_base = build_pipeline(et_base, use_sampler=False)


In [46]:
# [ET - Cell 2] OOF scores + chọn threshold theo F1

# Lấy OOF scores (xác suất class 1) trên TRAIN bằng CV
et_oof = get_oof_scores(et_pipe_base, X_train, y_train, cv5)

# Chọn threshold tối ưu theo F1 (ổn định & công bằng)
prec, rec, thr = precision_recall_curve(y_train, et_oof)
if len(thr) == 0:
    et_thr = 0.5
else:
    f1s = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
    et_thr = float(thr[np.argmax(f1s)])

# Tính metrics theo threshold đã chọn (OOF TRAIN)
et_m, et_cm = compute_metrics(y_train, et_oof, et_thr)

# Lưu kết quả baseline
baseline_results.append({
    "model": "ET",
    "phase": "baseline",
    "status": "OK",
    "thr": et_thr,
    **et_m
})

print("Chosen threshold (F1) =", round(et_thr, 4))
print("Predicted positive rate =", round(float((et_oof >= et_thr).mean()), 4))
display(pd.DataFrame([et_m]).round(4))


Chosen threshold (F1) = 0.3375
Predicted positive rate = 0.1231


,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier
0,0.9462,0.8565,0.6914,0.8616,0.7672,0.9799,0.8356,0.0544


In [47]:
# [ET - Cell 3] ROC-AUC + Classification report + Confusion Matrix + ROC (OOF TRAIN)

et_pred_oof = (et_oof >= et_thr).astype(int)

# ROC-AUC (OOF TRAIN)
et_roc_auc = roc_auc_score(y_train, et_oof)
print("ROC-AUC (OOF TRAIN) =", round(float(et_roc_auc), 4))

# Classification report
print(classification_report(y_train, et_pred_oof, digits=3))

# Confusion Matrix (Plotly cho đẹp)
if HAS_PLOTLY:
    cm = et_cm
    fig = go.Figure(
        data=go.Heatmap(
            z=cm,
            x=["Pred 0", "Pred 1"],
            y=["True 0", "True 1"],
            colorscale="Blues",
            zmin=0,
            zmax=int(cm.max())
        )
    )
    fig.update_layout(
        title="ET Baseline - Confusion Matrix (OOF TRAIN)",
        xaxis_title="Predicted",
        yaxis_title="True"
    )
    for i in range(2):
        for j in range(2):
            fig.add_annotation(
                x=j, y=i, text=str(cm[i, j]),
                showarrow=False, font=dict(size=16)
            )
    fig.show()
else:
    plot_cm(et_cm, title="ET Baseline - Confusion Matrix (OOF TRAIN)")

# ROC curve
plot_roc(y_train, et_oof, title="ET Baseline - ROC (OOF TRAIN)")


ROC-AUC (OOF TRAIN) = 0.9462
              precision    recall  f1-score   support

           0      0.946     0.980     0.963      7205
           1      0.862     0.691     0.767      1306

    accuracy                          0.936      8511
   macro avg      0.904     0.836     0.865      8511
weighted avg      0.933     0.936     0.933      8511



### ADA — AdaBoost (Baseline, OOF TRAIN)

In [48]:
# [ADA - Cell 1] Build baseline AdaBoost pipeline
# Lưu ý:
# - KHÔNG oversample AdaBoost
# - AdaBoost nhạy với noise/duplicate

from sklearn.ensemble import AdaBoostClassifier

ada_base = AdaBoostClassifier(
    n_estimators=300,
    learning_rate=0.05,
    random_state=SEED
)

# TẮT sampler cho AdaBoost
ada_pipe_base = build_pipeline(ada_base, use_sampler=False)


In [49]:
# [ADA - Cell 2] OOF scores + chọn threshold theo F1

# Lấy OOF scores (xác suất class 1) trên TRAIN bằng CV
ada_oof = get_oof_scores(ada_pipe_base, X_train, y_train, cv5)

# Chọn threshold tối ưu theo F1 (ổn định & công bằng)
prec, rec, thr = precision_recall_curve(y_train, ada_oof)
if len(thr) == 0:
    ada_thr = 0.5
else:
    f1s = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
    ada_thr = float(thr[np.argmax(f1s)])

# Tính metrics theo threshold đã chọn (OOF TRAIN)
ada_m, ada_cm = compute_metrics(y_train, ada_oof, ada_thr)

# Lưu kết quả baseline
baseline_results.append({
    "model": "ADA",
    "phase": "baseline",
    "status": "OK",
    "thr": ada_thr,
    **ada_m
})

print("Chosen threshold (F1) =", round(ada_thr, 4))
print("Predicted positive rate =", round(float((ada_oof >= ada_thr).mean()), 4))
display(pd.DataFrame([ada_m]).round(4))


Chosen threshold (F1) = 0.2559
Predicted positive rate = 0.3514


,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier
0,0.7126,0.316,0.621,0.2711,0.3775,0.6974,0.6592,0.1274


In [50]:
# [ADA - Cell 3] ROC-AUC + Classification report + Confusion Matrix + ROC (OOF TRAIN)

ada_pred_oof = (ada_oof >= ada_thr).astype(int)

# ROC-AUC (OOF TRAIN)
ada_roc_auc = roc_auc_score(y_train, ada_oof)
print("ROC-AUC (OOF TRAIN) =", round(float(ada_roc_auc), 4))

# Classification report
print(classification_report(y_train, ada_pred_oof, digits=3))

# Confusion Matrix (Plotly cho đẹp)
if HAS_PLOTLY:
    cm = ada_cm
    fig = go.Figure(
        data=go.Heatmap(
            z=cm,
            x=["Pred 0", "Pred 1"],
            y=["True 0", "True 1"],
            colorscale="Blues",
            zmin=0,
            zmax=int(cm.max())
        )
    )
    fig.update_layout(
        title="ADA Baseline - Confusion Matrix (OOF TRAIN)",
        xaxis_title="Predicted",
        yaxis_title="True"
    )
    for i in range(2):
        for j in range(2):
            fig.add_annotation(
                x=j, y=i, text=str(cm[i, j]),
                showarrow=False, font=dict(size=16)
            )
    fig.show()
else:
    plot_cm(ada_cm, title="ADA Baseline - Confusion Matrix (OOF TRAIN)")

# ROC curve
plot_roc(y_train, ada_oof, title="ADA Baseline - ROC (OOF TRAIN)")


ROC-AUC (OOF TRAIN) = 0.7126
              precision    recall  f1-score   support

           0      0.910     0.697     0.790      7205
           1      0.271     0.621     0.377      1306

    accuracy                          0.686      8511
   macro avg      0.591     0.659     0.584      8511
weighted avg      0.812     0.686     0.727      8511



### GB — Gradient Boosting (Baseline, OOF TRAIN)

In [51]:
# [GB - Cell 1] Build baseline GradientBoosting pipeline
# Lưu ý:
# - KHÔNG oversample GradientBoosting
# - GB học tốt non-linear nhưng nhạy với noise

from sklearn.ensemble import GradientBoostingClassifier

gb_base = GradientBoostingClassifier(
    random_state=SEED
)

# TẮT sampler cho GB
gb_pipe_base = build_pipeline(gb_base, use_sampler=False)


In [52]:
# [GB - Cell 2] OOF scores + chọn threshold theo F1

# Lấy OOF scores (xác suất class 1) trên TRAIN bằng CV
gb_oof = get_oof_scores(gb_pipe_base, X_train, y_train, cv5)

# Chọn threshold tối ưu theo F1
prec, rec, thr = precision_recall_curve(y_train, gb_oof)
if len(thr) == 0:
    gb_thr = 0.5
else:
    f1s = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
    gb_thr = float(thr[np.argmax(f1s)])

# Tính metrics theo threshold đã chọn (OOF TRAIN)
gb_m, gb_cm = compute_metrics(y_train, gb_oof, gb_thr)

# Lưu kết quả baseline
baseline_results.append({
    "model": "GB",
    "phase": "baseline",
    "status": "OK",
    "thr": gb_thr,
    **gb_m
})

print("Chosen threshold (F1) =", round(gb_thr, 4))
print("Predicted positive rate =", round(float((gb_oof >= gb_thr).mean()), 4))
display(pd.DataFrame([gb_m]).round(4))


Chosen threshold (F1) = 0.1987
Predicted positive rate = 0.2402


,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier
0,0.753,0.4365,0.5482,0.3503,0.4275,0.8157,0.682,0.1101


In [53]:
# [GB - Cell 3] ROC-AUC + Classification report + Confusion Matrix + ROC (OOF TRAIN)

gb_pred_oof = (gb_oof >= gb_thr).astype(int)

# ROC-AUC (OOF TRAIN)
gb_roc_auc = roc_auc_score(y_train, gb_oof)
print("ROC-AUC (OOF TRAIN) =", round(float(gb_roc_auc), 4))

# Classification report
print(classification_report(y_train, gb_pred_oof, digits=3))

# Confusion Matrix (Plotly cho đẹp)
if HAS_PLOTLY:
    cm = gb_cm
    fig = go.Figure(
        data=go.Heatmap(
            z=cm,
            x=["Pred 0", "Pred 1"],
            y=["True 0", "True 1"],
            colorscale="Blues",
            zmin=0,
            zmax=int(cm.max())
        )
    )
    fig.update_layout(
        title="GB Baseline - Confusion Matrix (OOF TRAIN)",
        xaxis_title="Predicted",
        yaxis_title="True"
    )
    for i in range(2):
        for j in range(2):
            fig.add_annotation(
                x=j, y=i, text=str(cm[i, j]),
                showarrow=False, font=dict(size=16)
            )
    fig.show()
else:
    plot_cm(gb_cm, title="GB Baseline - Confusion Matrix (OOF TRAIN)")

# ROC curve
plot_roc(y_train, gb_oof, title="GB Baseline - ROC (OOF TRAIN)")


ROC-AUC (OOF TRAIN) = 0.753
              precision    recall  f1-score   support

           0      0.909     0.816     0.860      7205
           1      0.350     0.548     0.427      1306

    accuracy                          0.775      8511
   macro avg      0.630     0.682     0.644      8511
weighted avg      0.823     0.775     0.793      8511



### HGB — HistGradientBoosting (Baseline, OOF TRAIN)

In [54]:
# [HGB - Cell 1] Build baseline HistGradientBoosting pipeline
# Lưu ý:
# - KHÔNG oversample HGB
# - HGB rất mạnh cho tabular, xử lý non-linear tốt

from sklearn.ensemble import HistGradientBoostingClassifier

hgb_base = HistGradientBoostingClassifier(
    random_state=SEED
)

# TẮT sampler cho HGB
hgb_pipe_base = build_pipeline(hgb_base, use_sampler=False)


In [55]:
# [HGB - Cell 2] OOF scores + chọn threshold theo F1

# Lấy OOF scores (xác suất class 1) trên TRAIN bằng CV
hgb_oof = get_oof_scores(hgb_pipe_base, X_train, y_train, cv5)

# Chọn threshold tối ưu theo F1
prec, rec, thr = precision_recall_curve(y_train, hgb_oof)
if len(thr) == 0:
    hgb_thr = 0.5
else:
    f1s = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
    hgb_thr = float(thr[np.argmax(f1s)])

# Tính metrics theo threshold đã chọn (OOF TRAIN)
hgb_m, hgb_cm = compute_metrics(y_train, hgb_oof, hgb_thr)

# Lưu kết quả baseline
baseline_results.append({
    "model": "HGB",
    "phase": "baseline",
    "status": "OK",
    "thr": hgb_thr,
    **hgb_m
})

print("Chosen threshold (F1) =", round(hgb_thr, 4))
print("Predicted positive rate =", round(float((hgb_oof >= hgb_thr).mean()), 4))
display(pd.DataFrame([hgb_m]).round(4))


Chosen threshold (F1) = 0.2682
Predicted positive rate = 0.1451


,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier
0,0.8621,0.649,0.5965,0.6308,0.6131,0.9367,0.7666,0.0863


In [56]:
# [HGB - Cell 3] ROC-AUC + Classification report + Confusion Matrix + ROC (OOF TRAIN)

hgb_pred_oof = (hgb_oof >= hgb_thr).astype(int)

# ROC-AUC (OOF TRAIN)
hgb_roc_auc = roc_auc_score(y_train, hgb_oof)
print("ROC-AUC (OOF TRAIN) =", round(float(hgb_roc_auc), 4))

# Classification report
print(classification_report(y_train, hgb_pred_oof, digits=3))

# Confusion Matrix (Plotly cho đẹp)
if HAS_PLOTLY:
    cm = hgb_cm
    fig = go.Figure(
        data=go.Heatmap(
            z=cm,
            x=["Pred 0", "Pred 1"],
            y=["True 0", "True 1"],
            colorscale="Blues",
            zmin=0,
            zmax=int(cm.max())
        )
    )
    fig.update_layout(
        title="HGB Baseline - Confusion Matrix (OOF TRAIN)",
        xaxis_title="Predicted",
        yaxis_title="True"
    )
    for i in range(2):
        for j in range(2):
            fig.add_annotation(
                x=j, y=i, text=str(cm[i, j]),
                showarrow=False, font=dict(size=16)
            )
    fig.show()
else:
    plot_cm(hgb_cm, title="HGB Baseline - Confusion Matrix (OOF TRAIN)")

# ROC curve
plot_roc(y_train, hgb_oof, title="HGB Baseline - ROC (OOF TRAIN)")


ROC-AUC (OOF TRAIN) = 0.8621
              precision    recall  f1-score   support

           0      0.928     0.937     0.932      7205
           1      0.631     0.596     0.613      1306

    accuracy                          0.885      8511
   macro avg      0.779     0.767     0.773      8511
weighted avg      0.882     0.885     0.883      8511



### XGB — XGBoost (Baseline, OOF TRAIN)

In [57]:
# [XGB - Cell 1] Build baseline XGBoost pipeline
# Lưu ý:
# - KHÔNG oversample XGB
# - XGB rất mạnh cho tabular, xử lý non-linear tốt

if not HAS_XGB:
    print("XGB: package not installed -> Skipped")
    baseline_results.append({
        "model": "XGB",
        "phase": "baseline",
        "status": "Skipped",
        "thr": np.nan
    })
else:
    xgb_base = XGBClassifier(
        n_estimators=400,
        max_depth=3,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        eval_metric="logloss",
        random_state=SEED,
        n_jobs=-1
    )

    # TẮT sampler cho XGB
    xgb_pipe_base = build_pipeline(xgb_base, use_sampler=False)


In [58]:
# [XGB - Cell 2] OOF scores + chọn threshold theo F1

if HAS_XGB:
    # OOF scores (CV validation)
    xgb_oof = get_oof_scores(xgb_pipe_base, X_train, y_train, cv5)

    # Chọn threshold tối ưu theo F1
    prec, rec, thr = precision_recall_curve(y_train, xgb_oof)
    if len(thr) == 0:
        xgb_thr = 0.5
    else:
        f1s = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
        xgb_thr = float(thr[np.argmax(f1s)])

    # Metrics (OOF TRAIN)
    xgb_m, xgb_cm = compute_metrics(y_train, xgb_oof, xgb_thr)

    baseline_results.append({
        "model": "XGB",
        "phase": "baseline",
        "status": "OK",
        "thr": xgb_thr,
        **xgb_m
    })

    print("Chosen threshold (F1) =", round(xgb_thr, 4))
    print("Predicted positive rate =", round(float((xgb_oof >= xgb_thr).mean()), 4))
    display(pd.DataFrame([xgb_m]).round(4))


Chosen threshold (F1) = 0.2393
Predicted positive rate = 0.1793


,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier
0,0.7772,0.478,0.5046,0.4318,0.4654,0.8797,0.6921,0.106


In [59]:
# [XGB - Cell 3] ROC-AUC + Classification report + Confusion Matrix + ROC (OOF TRAIN)

if HAS_XGB:
    xgb_pred_oof = (xgb_oof >= xgb_thr).astype(int)

    # ROC-AUC (OOF TRAIN)
    xgb_roc_auc = roc_auc_score(y_train, xgb_oof)
    print("ROC-AUC (OOF TRAIN) =", round(float(xgb_roc_auc), 4))

    # Classification report
    print(classification_report(y_train, xgb_pred_oof, digits=3))

    # Confusion Matrix (Plotly cho đẹp)
    if HAS_PLOTLY:
        cm = xgb_cm
        fig = go.Figure(
            data=go.Heatmap(
                z=cm,
                x=["Pred 0", "Pred 1"],
                y=["True 0", "True 1"],
                colorscale="Blues",
                zmin=0,
                zmax=int(cm.max())
            )
        )
        fig.update_layout(
            title="XGB Baseline - Confusion Matrix (OOF TRAIN)",
            xaxis_title="Predicted",
            yaxis_title="True"
        )
        for i in range(2):
            for j in range(2):
                fig.add_annotation(
                    x=j, y=i, text=str(cm[i, j]),
                    showarrow=False, font=dict(size=16)
                )
        fig.show()
    else:
        plot_cm(xgb_cm, title="XGB Baseline - Confusion Matrix (OOF TRAIN)")

    # ROC curve
    plot_roc(y_train, xgb_oof, title="XGB Baseline - ROC (OOF TRAIN)")


ROC-AUC (OOF TRAIN) = 0.7772
              precision    recall  f1-score   support

           0      0.907     0.880     0.893      7205
           1      0.432     0.505     0.465      1306

    accuracy                          0.822      8511
   macro avg      0.670     0.692     0.679      8511
weighted avg      0.834     0.822     0.828      8511



### LGBM — LightGBM (Baseline, OOF TRAIN)

In [60]:
# [LGBM - Cell 1] Build baseline LightGBM pipeline
# Lưu ý:
# - KHÔNG oversample LGBM
# - LGBM rất mạnh cho tabular, tốc độ nhanh

if not HAS_LGBM:
    print("LGBM: package not installed -> Skipped")
    baseline_results.append({
        "model": "LGBM",
        "phase": "baseline",
        "status": "Skipped",
        "thr": np.nan
    })
else:
    from lightgbm import LGBMClassifier

    lgbm_base = LGBMClassifier(
        n_estimators=400,
        learning_rate=0.05,
        random_state=SEED,
        n_jobs=-1
    )
    
    # TẮT sampler cho LGBM
    lgbm_pipe_base = build_pipeline(lgbm_base, use_sampler=False)


In [61]:
# [LGBM - Cell 2] OOF scores + chọn threshold theo F1

if HAS_LGBM:
    # OOF scores (CV validation)
    lgbm_oof = get_oof_scores(lgbm_pipe_base, X_train, y_train, cv5)

    # Chọn threshold tối ưu theo F1
    prec, rec, thr = precision_recall_curve(y_train, lgbm_oof)
    if len(thr) == 0:
        lgbm_thr = 0.5
    else:
        f1s = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
        lgbm_thr = float(thr[np.argmax(f1s)])

    # Metrics (OOF TRAIN)
    lgbm_m, lgbm_cm = compute_metrics(y_train, lgbm_oof, lgbm_thr)

    baseline_results.append({
        "model": "LGBM",
        "phase": "baseline",
        "status": "OK",
        "thr": lgbm_thr,
        **lgbm_m
    })
    
    print("Chosen threshold (F1) =", round(lgbm_thr, 4))
    print("Predicted positive rate =", round(float((lgbm_oof >= lgbm_thr).mean()), 4))
    display(pd.DataFrame([lgbm_m]).round(4))


Chosen threshold (F1) = 0.2468
Predicted positive rate = 0.1426


,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier
0,0.8879,0.7256,0.6524,0.7018,0.6762,0.9498,0.8011,0.0755


In [62]:
# [LGBM - Cell 3] ROC-AUC + Classification report + Confusion Matrix + ROC (OOF TRAIN)

if HAS_LGBM:
    lgbm_pred_oof = (lgbm_oof >= lgbm_thr).astype(int)

    # ROC-AUC (OOF TRAIN)
    lgbm_roc_auc = roc_auc_score(y_train, lgbm_oof)
    print("ROC-AUC (OOF TRAIN) =", round(float(lgbm_roc_auc), 4))

    # Classification report
    print(classification_report(y_train, lgbm_pred_oof, digits=3))

    # Confusion Matrix (Plotly cho đẹp)
    if HAS_PLOTLY:
        cm = lgbm_cm
        fig = go.Figure(
            data=go.Heatmap(
                z=cm,
                x=["Pred 0", "Pred 1"],
                y=["True 0", "True 1"],
                colorscale="Blues",
                zmin=0,
                zmax=int(cm.max())
            )
        )
        fig.update_layout(
            title="LGBM Baseline - Confusion Matrix (OOF TRAIN)",
            xaxis_title="Predicted",
            yaxis_title="True"
        )
        for i in range(2):
            for j in range(2):
                fig.add_annotation(
                    x=j, y=i, text=str(cm[i, j]),
                    showarrow=False, font=dict(size=16)
                )
        fig.show()
    else:
        plot_cm(lgbm_cm, title="LGBM Baseline - Confusion Matrix (OOF TRAIN)")

    # ROC curve
    plot_roc(y_train, lgbm_oof, title="LGBM Baseline - ROC (OOF TRAIN)")


ROC-AUC (OOF TRAIN) = 0.8879
              precision    recall  f1-score   support

           0      0.938     0.950     0.944      7205
           1      0.702     0.652     0.676      1306

    accuracy                          0.904      8511
   macro avg      0.820     0.801     0.810      8511
weighted avg      0.902     0.904     0.903      8511



### Tổng hợp baseline (OOF TRAIN)

**Ý nghĩa các metrics dùng để so sánh và xếp hạng mô hình (Baseline & Tuning)**

Trong bài toán dự đoán nguy cơ bệnh tim (TenYearCHD), dữ liệu bị **mất cân bằng lớp** (số ca không bệnh lớn hơn số ca bệnh). Vì vậy, **accuracy không được sử dụng**, thay vào đó là các metrics phản ánh tốt hơn khả năng **xếp hạng rủi ro** và **phát hiện ca bệnh**.

---

**ROC-AUC (Receiver Operating Characteristic – Area Under Curve)**  
- Đo khả năng mô hình **xếp hạng rủi ro** giữa người có bệnh và không bệnh.  
- Không phụ thuộc vào threshold và ít bị ảnh hưởng bởi mất cân bằng lớp.  
- Là **metric quan trọng nhất** trong pipeline này để so sánh và xếp hạng mô hình.

---

**PR-AUC (Average Precision / Precision–Recall AUC)**  
- Đánh giá hiệu quả của mô hình trên **lớp bệnh (positive class)**.  
- Nhạy với dữ liệu mất cân bằng hơn ROC-AUC.  
- Dùng để so sánh bổ sung khi các mô hình có ROC-AUC gần nhau.

---

**Recall (Sensitivity / True Positive Rate)**  
- Tỷ lệ bệnh nhân thật sự có bệnh được mô hình phát hiện.  
- Recall thấp đồng nghĩa với việc **bỏ sót bệnh nhân**, điều không mong muốn trong y tế.  
- Được ưu tiên trong các kịch bản screening.

---

**Precision (Positive Predictive Value)**  
- Trong các ca được dự đoán là có bệnh, tỷ lệ dự đoán đúng.  
- Precision thấp dẫn đến nhiều cảnh báo giả, làm tăng gánh nặng cho hệ thống y tế.

---

**F1-score**  
- Trung bình điều hoà giữa precision và recall.  
- Dùng để đánh giá mức cân bằng giữa hai yếu tố này, nhưng **không phải metric chính** trong pipeline.

---

**Specificity (True Negative Rate)**  
- Tỷ lệ người không bệnh được dự đoán đúng.  
- Cao specificity giúp giảm số lượng false positive.

---

**Balanced Accuracy**  
- Trung bình của recall (sensitivity) và specificity.  
- Phù hợp hơn accuracy khi dữ liệu mất cân bằng.

---

**Brier Score**  
- Đánh giá **chất lượng xác suất dự đoán** của mô hình.  
- Giá trị càng thấp thì xác suất dự đoán càng đáng tin cậy.  
- Quan trọng cho các bước **calibration** và ra quyết định dựa trên xác suất.

---

**Threshold (thr)**  
- Ngưỡng xác suất dùng để chuyển từ probability sang nhãn 0/1.  
- Threshold được chọn tuỳ theo mục tiêu (ưu tiên recall hoặc tối ưu F1).  
- **Không dùng để xếp hạng mô hình**, chỉ dùng sau khi đã chọn được mô hình tốt.

---

**Nguyên tắc xếp hạng mô hình trong pipeline**

Thứ tự ưu tiên khi so sánh các mô hình (baseline và sau tuning):
1. ROC-AUC  
2. PR-AUC  
3. Recall  

Các metric còn lại được dùng để tham khảo và phân tích sâu hơn.


In [63]:
# ===============================
# BASELINE SUMMARY + TOP MODELS
# ===============================

# Gom kết quả baseline
baseline_df = pd.DataFrame(baseline_results)

# Danh sách cột chuẩn mong muốn
cols = [
    "model",
    "status",
    "roc_auc",      # metric chính
    "pr_auc",
    "recall",
    "precision",
    "f1",
    "specificity",
    "bal_acc",
    "brier",
    "thr"
]

# Đảm bảo mọi cột đều tồn tại (kể cả model Skipped)
for c in cols:
    if c not in baseline_df.columns:
        baseline_df[c] = np.nan

# Giữ đúng thứ tự cột
baseline_df = baseline_df[cols]

# Sort chuẩn cho bài toán y tế:
# 1) Model OK trước
# 2) ROC-AUC (↓) — metric chính
# 3) PR-AUC (↓) — positive class
# 4) Recall (↓) — hạn chế bỏ sót bệnh
baseline_df = baseline_df.sort_values(
    by=["status", "roc_auc", "pr_auc", "recall"],
    ascending=[True, False, False, False]
)

# Hiển thị bảng baseline
print("=== BASELINE RESULTS (OOF-CV) ===")
display(baseline_df.round(4))

# ===============================
# CHỌN TOP 5 MODELS ĐỂ TUNING
# ===============================

top5_models = baseline_df.query("status == 'OK'").head(5)

print("=== TOP 5 MODELS FOR TUNING ===")
display(top5_models.round(4))

# Lấy danh sách tên model (dùng cho bước tuning)
TOP5_MODEL_NAMES = top5_models["model"].tolist()
print("Top 5 model names:", TOP5_MODEL_NAMES)


=== BASELINE RESULTS (OOF-CV) ===


,model,status,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier,thr
6,ET,OK,0.9462,0.8565,0.6914,0.8616,0.7672,0.9799,0.8356,0.0544,0.3375
5,RF,OK,0.9381,0.8435,0.7175,0.8043,0.7584,0.9684,0.8429,0.0659,0.2800
11,LGBM,OK,0.8879,0.7256,0.6524,0.7018,0.6762,0.9498,0.8011,0.0755,0.2468
9,HGB,OK,0.8621,0.6490,0.5965,0.6308,0.6131,0.9367,0.7666,0.0863,0.2682
10,XGB,OK,0.7772,0.4780,0.5046,0.4318,0.4654,0.8797,0.6921,0.1060,0.2393
2,KNN,OK,0.7716,0.3552,0.5360,0.3610,0.4314,0.8280,0.6820,0.1127,0.2667
3,SVC,OK,0.7713,0.3720,0.5260,0.4003,0.4547,0.8572,0.6916,0.1119,0.2600
8,GB,OK,0.7530,0.4365,0.5482,0.3503,0.4275,0.8157,0.6820,0.1101,0.1987
4,DT,OK,0.7262,0.3646,0.5306,0.5513,0.5408,0.9217,0.7262,0.1383,1.0000
0,LR,OK,0.7216,0.3419,0.6133,0.2822,0.3866,0.7173,0.6653,0.2119,0.5323


=== TOP 5 MODELS FOR TUNING ===


,model,status,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier,thr
6,ET,OK,0.9462,0.8565,0.6914,0.8616,0.7672,0.9799,0.8356,0.0544,0.3375
5,RF,OK,0.9381,0.8435,0.7175,0.8043,0.7584,0.9684,0.8429,0.0659,0.2800
11,LGBM,OK,0.8879,0.7256,0.6524,0.7018,0.6762,0.9498,0.8011,0.0755,0.2468
9,HGB,OK,0.8621,0.6490,0.5965,0.6308,0.6131,0.9367,0.7666,0.0863,0.2682
10,XGB,OK,0.7772,0.4780,0.5046,0.4318,0.4654,0.8797,0.6921,0.1060,0.2393


Top 5 model names: ['ET', 'RF', 'LGBM', 'HGB', 'XGB']


## Tuning: RandomizedSearchCV (sau khi baseline xong)

In [64]:
# =========================
# GLOBAL SETUP (1 cell dùng chung cho mọi model tuning)
# =========================

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import precision_recall_curve

# ---- CONFIG ----
SEED = 42
N_ITER = 25  # 20–30 hợp lý cho máy cá nhân

cv5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# ---- SEARCH SCORING (multi-metric) ----
SCORING_OPT = {"pr_auc": "average_precision", "roc_auc": "roc_auc"}
REFIT_OPT = "pr_auc"

print(f"SEED={SEED} | N_ITER={N_ITER} | CV=5 | scoring={list(SCORING_OPT.keys())} | refit={REFIT_OPT}")

# ---- STORAGE (nếu chưa có) ----
if "best_params" not in globals():
    best_params = {}
if "tuned_results" not in globals():
    tuned_results = []

# ---- THRESHOLD PICKER (stable) ----
def pick_threshold_by_target_recall(
    y_true,
    scores,
    target_recall=0.85,
    min_precision=0.30
):
    """
    Chọn threshold ổn định:
    - Ưu tiên đạt recall >= target_recall
    - Đồng thời precision >= min_precision
    - Tránh threshold quá thấp (all-1)
    """
    prec, rec, thr = precision_recall_curve(y_true, scores)

    if len(thr) == 0:
        return 0.5

    prec2, rec2, thr2 = prec[:-1], rec[:-1], thr
    mask = (rec2 >= target_recall) & (prec2 >= min_precision)

    if mask.any():
        return float(thr2[mask][-1])

    idx = np.argmin(np.abs(rec2 - target_recall))
    return float(thr2[idx])

# ---- TEMPLATE SEARCH + OOF + THRESHOLD + METRICS ----
def run_search_oof_threshold(
    model_name: str,
    base_estimator,
    param_space: dict,      # param_distributions
    use_sampler: bool,
    X_train,
    y_train,
    cv,
    n_iter: int = None,
    target_recall: float = 0.85,
    min_precision: float = 0.30,
    topk: int = 10,
    verbose: int = 1,
    random_state: int = None
):
    """
    Template CHUNG cho mọi model:
    1) RandomizedSearchCV (PR-AUC optimize + log ROC-AUC)
    2) Fit lại best
    3) OOF scores
    4) Pick threshold theo target recall
    5) Compute metrics + append tuned_results
    """

    if n_iter is None:
        n_iter = N_ITER
    if random_state is None:
        random_state = SEED

    pipe = build_pipeline(base_estimator, use_sampler=use_sampler)

    search = RandomizedSearchCV(
        estimator=pipe,
        param_distributions=param_space,
        n_iter=n_iter,
        scoring=SCORING_OPT,
        refit=REFIT_OPT,
        cv=cv,
        n_jobs=-1,
        verbose=verbose,
        random_state=random_state,
        return_train_score=False
    )

    search.fit(X_train, y_train)
    best_params[model_name] = search.best_params_

    res = pd.DataFrame(search.cv_results_)
    best_row = res[res["rank_test_pr_auc"] == 1].iloc[0]

    print(f"\n===== {model_name} | RANDOM SEARCH DONE =====")
    print(
        "Best CV:",
        "PR-AUC =", round(float(best_row["mean_test_pr_auc"]), 5),
        "| ROC-AUC =", round(float(best_row["mean_test_roc_auc"]), 5)
    )

    cols_show = [
        "rank_test_pr_auc", "mean_test_pr_auc", "std_test_pr_auc",
        "mean_test_roc_auc", "std_test_roc_auc", "params"
    ]
    display(res.sort_values("rank_test_pr_auc").head(topk)[cols_show])

    # ----- OOF + threshold + metrics -----
    tuned_estimator = base_estimator.__class__(**base_estimator.get_params())
    tuned_pipe = build_pipeline(tuned_estimator, use_sampler=use_sampler)
    tuned_pipe.set_params(**best_params[model_name])

    oof_scores = get_oof_scores(tuned_pipe, X_train, y_train, cv)

    thr = pick_threshold_by_target_recall(
        y_train, oof_scores,
        target_recall=target_recall,
        min_precision=min_precision
    )

    m, cm = compute_metrics(y_train, oof_scores, thr)
    tn, fp, fn, tp = cm.ravel()

    print("Chosen threshold =", round(thr, 4))
    print("OOF ROC-AUC =", round(m["roc_auc"], 4), "| PR-AUC =", round(m["pr_auc"], 4))
    print("Score quantiles:", np.quantile(oof_scores, [0.01, 0.05, 0.1, 0.2, 0.5]))

    tuned_results.append({
        "model": model_name,
        "phase": "tuned",
        "status": "OK",
        "thr": thr,
        **m
    })

    display(pd.DataFrame([{**m, "TP": tp, "FP": fp, "TN": tn, "FN": fn}]).round(4))

    return {
        "search": search,
        "cv_results": res,
        "oof_scores": oof_scores,
        "thr": thr,
        "metrics": m,
        "cm": cm
    }


SEED=42 | N_ITER=25 | CV=5 | scoring=['pr_auc', 'roc_auc'] | refit=pr_auc


### RF — Random Forest (Tuning, RandomizedSearchCV)

**Tuning Random Forest (RF)**

Ở bước này, Random Forest được tuning hyperparameters để cải thiện hiệu năng so với baseline trước khi so sánh lại với các mô hình khác.

- **RandomizedSearchCV** được dùng để thử ngẫu nhiên nhiều cấu hình hyperparameter với chi phí tính toán hợp lý.
- **PR-AUC (average precision)** được sử dụng làm metric trong tuning vì dữ liệu mất cân bằng và lớp quan trọng là **CHD = 1**.
- **Không sử dụng oversampling** cho Random Forest vì đây là mô hình tree-based và đã xử lý imbalance thông qua `class_weight`.
- Sau tuning, mô hình được đánh giá lại bằng **OOF-CV** để:
  - tính ROC-AUC, PR-AUC
  - chọn threshold ưu tiên recall
  - phân tích confusion matrix và ROC curve

**Mục tiêu:** so sánh RF (tuned) với các mô hình khác để chọn ra mô hình tốt nhất cho bước calibration và final evaluation.


In [65]:
from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd

# -------------------------------------------------
# RF: param space
# -------------------------------------------------
rf_param_dist = {
    "model__n_estimators": randint(400, 900),
    "model__max_depth": randint(18, 32),
    "model__min_samples_leaf": randint(1, 6),
    "model__max_features": ["sqrt", "log2", None],
}

# đảm bảo best_params có tồn tại
if "best_params" not in globals():
    best_params = {}

# -------------------------------------------------
# RF: RandomizedSearchCV (opt PR-AUC, log ROC-AUC)
# -------------------------------------------------
rf_base = RandomForestClassifier(
    class_weight="balanced",
    random_state=SEED,
    n_jobs=-1
)

rf_pipe_for_search = build_pipeline(rf_base, use_sampler=False)

SCORING_OPT = {"pr_auc": "average_precision", "roc_auc": "roc_auc"}
REFIT_OPT = "pr_auc"

search_rf = RandomizedSearchCV(
    estimator=rf_pipe_for_search,
    param_distributions=rf_param_dist,
    n_iter=N_ITER,                 # hoặc ghi trực tiếp: 25
    scoring=SCORING_OPT,
    refit=REFIT_OPT,
    cv=cv5,
    n_jobs=-1,
    random_state=SEED,
    verbose=1,
    return_train_score=False
)

search_rf.fit(X_train, y_train)

best_params["RF"] = search_rf.best_params_

# -------------------------------------------------
# Report: best CV + top10
# -------------------------------------------------
res_rf = pd.DataFrame(search_rf.cv_results_)

print(f"Total candidates tried = {len(res_rf)} | Total fits ≈ {len(res_rf) * cv5.get_n_splits()}")

best_row_rf = res_rf.loc[res_rf["rank_test_pr_auc"] == 1].iloc[0]

print(
    "Best CV:",
    "PR-AUC =", round(float(best_row_rf["mean_test_pr_auc"]), 5),
    "| ROC-AUC =", round(float(best_row_rf["mean_test_roc_auc"]), 5)
)

cols_show = [
    "rank_test_pr_auc", "mean_test_pr_auc", "std_test_pr_auc",
    "mean_test_roc_auc", "std_test_roc_auc", "params"
]

print("\nTop 10 configs (by PR-AUC):")
display(res_rf.sort_values("rank_test_pr_auc").head(10)[cols_show])


Fitting 5 folds for each of 25 candidates, totalling 125 fits
Total candidates tried = 25 | Total fits ≈ 125
Best CV: PR-AUC = 0.84599 | ROC-AUC = 0.93863

Top 10 configs (by PR-AUC):


,rank_test_pr_auc,mean_test_pr_auc,std_test_pr_auc,mean_test_roc_auc,std_test_roc_auc,params
5,1,0.845988,0.018197,0.938627,0.004478,"{'model__max_depth': 29, 'model__max_features'..."
24,2,0.844997,0.018982,0.938297,0.004476,"{'model__max_depth': 30, 'model__max_features'..."
23,3,0.844914,0.018650,0.938655,0.004585,"{'model__max_depth': 29, 'model__max_features'..."
22,4,0.840364,0.017965,0.936060,0.004388,"{'model__max_depth': 22, 'model__max_features'..."
1,5,0.803866,0.022823,0.920462,0.004026,"{'model__max_depth': 30, 'model__max_features'..."
19,6,0.803745,0.023281,0.920363,0.004257,"{'model__max_depth': 29, 'model__max_features'..."
12,7,0.803644,0.023368,0.920144,0.004546,"{'model__max_depth': 29, 'model__max_features'..."
17,8,0.803590,0.023658,0.920042,0.004622,"{'model__max_depth': 31, 'model__max_features'..."
13,9,0.803337,0.023422,0.920222,0.004137,"{'model__max_depth': 27, 'model__max_features'..."
4,10,0.803096,0.023760,0.919743,0.004774,"{'model__max_depth': 25, 'model__max_features'..."


In [66]:
print("Best params =", best_params["RF"])

Best params = {'model__max_depth': 29, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 1, 'model__n_estimators': 859}


### ET — Extra Trees (Tuning, RandomizedSearchCV)

In [67]:
from scipy.stats import randint
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import numpy as np

# -------------------------------------------------
# ET: param space
# -------------------------------------------------
et_param_dist = {
    "model__n_estimators": randint(200, 900),
    "model__max_depth": randint(3, 31),
    "model__min_samples_leaf": randint(1, 11),
    "model__max_features": ["sqrt", "log2", None],
}

# đảm bảo best_params / tuned_results có tồn tại
if "best_params" not in globals():
    best_params = {}
if "tuned_results" not in globals():
    tuned_results = []

# -------------------------------------------------
# ET: RandomizedSearchCV (opt PR-AUC, log ROC-AUC)
# -------------------------------------------------
et_base = ExtraTreesClassifier(
    class_weight="balanced",
    random_state=SEED,
    n_jobs=-1
)

et_pipe_for_search = build_pipeline(et_base, use_sampler=False)

SCORING_OPT = {"pr_auc": "average_precision", "roc_auc": "roc_auc"}
REFIT_OPT = "pr_auc"

search_et = RandomizedSearchCV(
    estimator=et_pipe_for_search,
    param_distributions=et_param_dist,
    n_iter=N_ITER,                 # muốn nhẹ hơn thì giảm (vd 15/20)
    scoring=SCORING_OPT,
    refit=REFIT_OPT,
    cv=cv5,
    n_jobs=-1,
    random_state=SEED,
    verbose=1,
    return_train_score=False
)

search_et.fit(X_train, y_train)

best_params["ET"] = search_et.best_params_

# -------------------------------------------------
# Report: best CV + top10
# -------------------------------------------------
res_et = pd.DataFrame(search_et.cv_results_)
print(f"Total candidates tried = {len(res_et)} | Total fits ≈ {len(res_et) * cv5.get_n_splits()}")

best_row_et = res_et.loc[res_et["rank_test_pr_auc"] == 1].iloc[0]
print(
    "Best CV:",
    "PR-AUC =", round(float(best_row_et["mean_test_pr_auc"]), 5),
    "| ROC-AUC =", round(float(best_row_et["mean_test_roc_auc"]), 5)
)

cols_show = [
    "rank_test_pr_auc", "mean_test_pr_auc", "std_test_pr_auc",
    "mean_test_roc_auc", "std_test_roc_auc", "params"
]
print("\nTop 10 configs (by PR-AUC):")
display(res_et.sort_values("rank_test_pr_auc").head(10)[cols_show])

# -------------------------------------------------
# ET: OOF + threshold + metrics
# -------------------------------------------------
et_tuned = ExtraTreesClassifier(
    class_weight="balanced",
    random_state=SEED,
    n_jobs=-1
)

et_pipe_tuned = build_pipeline(et_tuned, use_sampler=False)
et_pipe_tuned.set_params(**best_params["ET"])

et_oof_t = get_oof_scores(et_pipe_tuned, X_train, y_train, cv5)

et_thr_t = pick_threshold_by_target_recall(
    y_train,
    et_oof_t,
    target_recall=0.85,
    min_precision=0.30
)

et_m_t, et_cm_t = compute_metrics(y_train, et_oof_t, et_thr_t)
tn, fp, fn, tp = et_cm_t.ravel()

et_m_t_full = {**et_m_t, "TP": tp, "FP": fp, "TN": tn, "FN": fn}

tuned_results.append({
    "model": "ET",
    "phase": "tuned",
    "status": "OK",
    "thr": et_thr_t,
    **et_m_t
})

print("\nChosen threshold =", round(et_thr_t, 4))
print(
    "OOF ROC-AUC =", round(et_m_t["roc_auc"], 4),
    "| PR-AUC =", round(et_m_t["pr_auc"], 4)
)
print("Score quantiles:", np.quantile(et_oof_t, [0.01, 0.05, 0.1, 0.2, 0.5]))

print("\nOOF metrics + confusion:")
display(pd.DataFrame([et_m_t_full]).round(4))

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Total candidates tried = 25 | Total fits ≈ 125
Best CV: PR-AUC = 0.85672 | ROC-AUC = 0.94566

Top 10 configs (by PR-AUC):


,rank_test_pr_auc,mean_test_pr_auc,std_test_pr_auc,mean_test_roc_auc,std_test_roc_auc,params
5,1,0.856720,0.012424,0.945664,0.004941,"{'model__max_depth': 30, 'model__max_features'..."
8,2,0.812645,0.018200,0.917253,0.006053,"{'model__max_depth': 30, 'model__max_features'..."
10,3,0.762781,0.020798,0.898432,0.007304,"{'model__max_depth': 21, 'model__max_features'..."
22,4,0.759784,0.024308,0.891237,0.007457,"{'model__max_depth': 17, 'model__max_features'..."
24,5,0.756035,0.032565,0.890607,0.009813,"{'model__max_depth': 13, 'model__max_features'..."
11,6,0.729645,0.021434,0.885247,0.005435,"{'model__max_depth': 20, 'model__max_features'..."
18,7,0.681366,0.026968,0.875430,0.008000,"{'model__max_depth': 18, 'model__max_features'..."
13,8,0.681252,0.027133,0.875946,0.008405,"{'model__max_depth': 17, 'model__max_features'..."
9,9,0.643818,0.023793,0.857893,0.007345,"{'model__max_depth': 22, 'model__max_features'..."
2,10,0.642442,0.029475,0.861814,0.008863,"{'model__max_depth': 13, 'model__max_features'..."



Chosen threshold = 0.1801
OOF ROC-AUC = 0.9456 | PR-AUC = 0.8565
Score quantiles: [0.         0.00455235 0.00781249 0.01696104 0.06069803]

OOF metrics + confusion:


,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier,TP,FP,TN,FN
0,0.9456,0.8565,0.8507,0.5863,0.6942,0.8912,0.8709,0.0547,1111,784,6421,195


### HGB — HistGradientBoosting (Tuning, RandomizedSearchCV)

In [68]:
from scipy.stats import randint, loguniform
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import numpy as np

# đảm bảo best_params / tuned_results có tồn tại
if "best_params" not in globals():
    best_params = {}
if "tuned_results" not in globals():
    tuned_results = []

# -------------------------------------------------
# HGB: param space
# -------------------------------------------------
HGB_N_ITER = 80  # 60/80/120 tuỳ máy

hgb_param_dist = {
    "model__max_iter": randint(200, 1200),
    "model__learning_rate": loguniform(1e-3, 0.2),
    "model__max_leaf_nodes": randint(15, 255),
    "model__max_depth": [None, 3, 5, 7, 9],
    "model__min_samples_leaf": randint(10, 300),
    "model__l2_regularization": loguniform(1e-8, 1e-1),
    "model__max_bins": randint(64, 256),
}

# -------------------------------------------------
# HGB: RandomizedSearchCV (opt PR-AUC, log ROC-AUC)
# -------------------------------------------------
hgb_base = HistGradientBoostingClassifier(random_state=SEED)
hgb_pipe_for_search = build_pipeline(hgb_base, use_sampler=False)

SCORING_OPT = {"pr_auc": "average_precision", "roc_auc": "roc_auc"}
REFIT_OPT = "pr_auc"

search_hgb = RandomizedSearchCV(
    estimator=hgb_pipe_for_search,
    param_distributions=hgb_param_dist,
    n_iter=HGB_N_ITER,
    scoring=SCORING_OPT,
    refit=REFIT_OPT,
    cv=cv5,
    n_jobs=-1,
    random_state=SEED,
    verbose=2,
    return_train_score=False
)

search_hgb.fit(X_train, y_train)

best_params["HGB"] = search_hgb.best_params_

# -------------------------------------------------
# Report: best CV + top10
# -------------------------------------------------
res_hgb = pd.DataFrame(search_hgb.cv_results_)
print(f"Total candidates tried = {len(res_hgb)} | Total fits ≈ {len(res_hgb) * cv5.get_n_splits()}")

best_row_hgb = res_hgb.loc[res_hgb["rank_test_pr_auc"] == 1].iloc[0]
print(
    "Best CV:",
    "PR-AUC =", round(float(best_row_hgb["mean_test_pr_auc"]), 5),
    "| ROC-AUC =", round(float(best_row_hgb["mean_test_roc_auc"]), 5)
)

cols_show = [
    "rank_test_pr_auc","mean_test_pr_auc","std_test_pr_auc",
    "mean_test_roc_auc","std_test_roc_auc","params"
]
print("\nTop 10 configs (by PR-AUC):")
display(res_hgb.sort_values("rank_test_pr_auc").head(10)[cols_show])

# -------------------------------------------------
# HGB: OOF + threshold + metrics
# -------------------------------------------------
hgb_tuned = HistGradientBoostingClassifier(random_state=SEED)
hgb_pipe_tuned = build_pipeline(hgb_tuned, use_sampler=False)
hgb_pipe_tuned.set_params(**best_params["HGB"])

hgb_oof_t = get_oof_scores(hgb_pipe_tuned, X_train, y_train, cv5)

hgb_thr_t = pick_threshold_by_target_recall(
    y_train,
    hgb_oof_t,
    target_recall=0.85,
    min_precision=0.30
)

hgb_m_t, hgb_cm_t = compute_metrics(y_train, hgb_oof_t, hgb_thr_t)
tn, fp, fn, tp = hgb_cm_t.ravel()

hgb_m_t_full = {**hgb_m_t, "TP": tp, "FP": fp, "TN": tn, "FN": fn}

tuned_results.append({
    "model": "HGB",
    "phase": "tuned",
    "status": "OK",
    "thr": hgb_thr_t,
    **hgb_m_t
})

print("\nChosen threshold =", round(hgb_thr_t, 4))
print(
    "OOF ROC-AUC =", round(hgb_m_t["roc_auc"], 4),
    "| PR-AUC =", round(hgb_m_t["pr_auc"], 4)
)
print("Score quantiles:", np.quantile(hgb_oof_t, [0.01, 0.05, 0.1, 0.2, 0.5]))

print("\nOOF metrics + confusion:")
display(pd.DataFrame([hgb_m_t_full]).round(4))


Fitting 5 folds for each of 80 candidates, totalling 400 fits
Total candidates tried = 80 | Total fits ≈ 400
Best CV: PR-AUC = 0.83233 | ROC-AUC = 0.92551

Top 10 configs (by PR-AUC):


,rank_test_pr_auc,mean_test_pr_auc,std_test_pr_auc,mean_test_roc_auc,std_test_roc_auc,params
61,1,0.832329,0.016735,0.925508,0.005671,{'model__l2_regularization': 0.002205722945374...
25,2,0.824657,0.024508,0.920632,0.009088,{'model__l2_regularization': 2.30837413695962e...
36,3,0.817935,0.023963,0.917450,0.009483,{'model__l2_regularization': 0.019044598430332...
26,4,0.788429,0.029681,0.908485,0.006459,{'model__l2_regularization': 5.34727401833469e...
27,5,0.771296,0.024381,0.898214,0.008939,{'model__l2_regularization': 7.369993416775703...
45,6,0.769131,0.024255,0.897353,0.008837,{'model__l2_regularization': 0.000101402178224...
66,7,0.754743,0.027107,0.891940,0.010217,{'model__l2_regularization': 5.0433425224965e-...
14,8,0.734688,0.028036,0.889913,0.008579,{'model__l2_regularization': 0.030486400425112...
3,9,0.719229,0.026646,0.883661,0.006002,{'model__l2_regularization': 0.088202504091702...
42,10,0.717442,0.029442,0.886741,0.006691,{'model__l2_regularization': 0.000261800445943...



Chosen threshold = 0.0021
OOF ROC-AUC = 0.9253 | PR-AUC = 0.8316
Score quantiles: [3.32573365e-08 2.71249175e-07 9.33194160e-07 4.92465674e-06
 9.70717418e-05]

OOF metrics + confusion:


,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier,TP,FP,TN,FN
0,0.9253,0.8316,0.8507,0.5486,0.6671,0.8731,0.8619,0.0599,1111,914,6291,195


### XGB — XGBoost (Tuning, RandomizedSearchCV)

In [69]:
from scipy.stats import randint, uniform, loguniform
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import numpy as np

# đảm bảo best_params / tuned_results có tồn tại
if "best_params" not in globals():
    best_params = {}
if "tuned_results" not in globals():
    tuned_results = []

if not HAS_XGB:
    print("XGB: package not installed -> Skipped tuning")
    tuned_results.append({"model": "XGB", "phase": "tuned", "status": "Skipped", "thr": np.nan})
else:
    # -------------------------------------------------
    # XGB: param space
    # -------------------------------------------------
    XGB_N_ITER = 60  # 30/60/100 tuỳ máy

    xgb_param_dist = {
        "model__n_estimators": randint(300, 1200),
        "model__max_depth": randint(2, 8),
        "model__learning_rate": loguniform(1e-3, 0.2),
        "model__subsample": uniform(0.6, 0.4),
        "model__colsample_bytree": uniform(0.6, 0.4),
        "model__min_child_weight": randint(1, 15),
        "model__gamma": loguniform(1e-8, 1.0),
    }

    # -------------------------------------------------
    # XGB: RandomizedSearchCV (opt PR-AUC, log ROC-AUC)
    # -------------------------------------------------
    xgb_base = XGBClassifier(
        random_state=SEED,
        n_jobs=-1,
        eval_metric="logloss",
        tree_method="hist"   # đổi "gpu_hist" nếu bạn setup GPU OK
    )

    xgb_pipe_for_search = build_pipeline(xgb_base, use_sampler=False)

    SCORING_OPT = {"pr_auc": "average_precision", "roc_auc": "roc_auc"}
    REFIT_OPT = "pr_auc"

    search_xgb = RandomizedSearchCV(
        estimator=xgb_pipe_for_search,
        param_distributions=xgb_param_dist,
        n_iter=XGB_N_ITER,
        scoring=SCORING_OPT,
        refit=REFIT_OPT,
        cv=cv5,
        n_jobs=-1,
        random_state=SEED,
        verbose=1,
        return_train_score=False
    )

    search_xgb.fit(X_train, y_train)

    best_params["XGB"] = search_xgb.best_params_

    # -------------------------------------------------
    # Report: best CV + top10
    # -------------------------------------------------
    res_xgb = pd.DataFrame(search_xgb.cv_results_)
    print(f"Total candidates tried = {len(res_xgb)} | Total fits ≈ {len(res_xgb) * cv5.get_n_splits()}")

    best_row_xgb = res_xgb.loc[res_xgb["rank_test_pr_auc"] == 1].iloc[0]
    print(
        "Best CV:",
        "PR-AUC =", round(float(best_row_xgb["mean_test_pr_auc"]), 5),
        "| ROC-AUC =", round(float(best_row_xgb["mean_test_roc_auc"]), 5)
    )

    cols_show = [
        "rank_test_pr_auc", "mean_test_pr_auc", "std_test_pr_auc",
        "mean_test_roc_auc", "std_test_roc_auc", "params"
    ]
    print("\nTop 10 configs (by PR-AUC):")
    display(res_xgb.sort_values("rank_test_pr_auc").head(10)[cols_show])

    # -------------------------------------------------
    # XGB: OOF + threshold + metrics
    # -------------------------------------------------
    xgb_tuned = XGBClassifier(
        random_state=SEED,
        n_jobs=-1,
        eval_metric="logloss",
        tree_method="hist"
    )

    xgb_pipe_tuned = build_pipeline(xgb_tuned, use_sampler=False)
    xgb_pipe_tuned.set_params(**best_params["XGB"])

    xgb_oof_t = get_oof_scores(xgb_pipe_tuned, X_train, y_train, cv5)

    xgb_thr_t = pick_threshold_by_target_recall(
        y_train,
        xgb_oof_t,
        target_recall=0.85,
        min_precision=0.30
    )

    xgb_m_t, xgb_cm_t = compute_metrics(y_train, xgb_oof_t, xgb_thr_t)
    tn, fp, fn, tp = xgb_cm_t.ravel()

    xgb_m_t_full = {**xgb_m_t, "TP": tp, "FP": fp, "TN": tn, "FN": fn}

    tuned_results.append({
        "model": "XGB",
        "phase": "tuned",
        "status": "OK",
        "thr": xgb_thr_t,
        **xgb_m_t
    })

    print("\nChosen threshold =", round(xgb_thr_t, 4))
    print(
        "OOF ROC-AUC =", round(xgb_m_t["roc_auc"], 4),
        "| PR-AUC =", round(xgb_m_t["pr_auc"], 4)
    )
    print("Score quantiles:", np.quantile(xgb_oof_t, [0.01, 0.05, 0.1, 0.2, 0.5]))

    print("\nOOF metrics + confusion:")
    display(pd.DataFrame([xgb_m_t_full]).round(4))


Fitting 5 folds for each of 60 candidates, totalling 300 fits
Total candidates tried = 60 | Total fits ≈ 300
Best CV: PR-AUC = 0.72524 | ROC-AUC = 0.88629

Top 10 configs (by PR-AUC):


,rank_test_pr_auc,mean_test_pr_auc,std_test_pr_auc,mean_test_roc_auc,std_test_roc_auc,params
33,1,0.725237,0.030694,0.886294,0.009698,{'model__colsample_bytree': 0.8801431319891084...
0,2,0.698612,0.027046,0.878474,0.007048,"{'model__colsample_bytree': 0.749816047538945,..."
38,3,0.685155,0.021021,0.868738,0.011461,{'model__colsample_bytree': 0.7001847274422337...
8,4,0.684738,0.028252,0.869814,0.012135,{'model__colsample_bytree': 0.8270801311279966...
29,5,0.675251,0.025557,0.867005,0.009082,{'model__colsample_bytree': 0.6880964190262193...
26,6,0.670353,0.029495,0.865659,0.011622,{'model__colsample_bytree': 0.6579579488364892...
45,7,0.663246,0.030633,0.866375,0.008828,{'model__colsample_bytree': 0.9394679179698697...
23,8,0.628941,0.028284,0.848809,0.010636,{'model__colsample_bytree': 0.6673164168691722...
43,9,0.611834,0.030121,0.839860,0.013634,{'model__colsample_bytree': 0.6677970986744369...
20,10,0.607211,0.028371,0.844214,0.011507,{'model__colsample_bytree': 0.9570235993959911...



Chosen threshold = 0.0448
OOF ROC-AUC = 0.8862 | PR-AUC = 0.7248
Score quantiles: [0.0001246  0.0005687  0.00130535 0.00345051 0.02136761]

OOF metrics + confusion:


,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier,TP,FP,TN,FN
0,0.8862,0.7248,0.8507,0.3571,0.5031,0.7224,0.7866,0.0719,1111,2000,5205,195


### LGBM — LightGBM (Tuning, RandomizedSearchCV)

In [70]:
# =========================
# LGBM (ONE CELL ALL-IN-ONE):
# - RandomizedSearchCV tuning (optimize ROC-AUC, still report PR-AUC)
# - Print Top configs (by ROC-AUC & PR-AUC)
# - OOF scores + pick threshold (target recall/min precision)
# - Print: chosen threshold, OOF AUC/PR-AUC, score quantiles
# - Show OOF metrics + confusion table
# =========================

import numpy as np
import pandas as pd
from scipy.stats import randint, uniform, loguniform
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from lightgbm import LGBMClassifier

# ---- helper: show top configs ----
def show_top_configs(search, topk=10, metric="roc_auc"):
    df = pd.DataFrame(search.cv_results_)
    sort_col = f"mean_test_{metric}" if f"mean_test_{metric}" in df.columns else "mean_test_score"
    std_col  = f"std_test_{metric}"  if f"std_test_{metric}"  in df.columns else "std_test_score"
    rank_col = f"rank_test_{metric}" if f"rank_test_{metric}" in df.columns else "rank_test_score"

    base = df[[rank_col, sort_col, std_col, "params"]].sort_values(sort_col, ascending=False).head(topk).copy()
    params_df = pd.json_normalize(base["params"])
    out = pd.concat([base.drop(columns=["params"]).reset_index(drop=True),
                     params_df.reset_index(drop=True)], axis=1)
    display(out.round(5))

# ---- config ----
LGBM_N_ITER = 80
cv_lgbm = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

pos = int((y_train == 1).sum())
neg = int((y_train == 0).sum())
spw = neg / max(pos, 1)

# ---- model + pipeline (dùng helper của notebook) ----
lgbm_base = LGBMClassifier(
    objective="binary",
    random_state=SEED,
    n_jobs=1,
    force_col_wise=True,
    verbose=-1
)
lgbm_pipe = build_pipeline(lgbm_base, use_sampler=False)

# ---- param space (ổn định hơn để giảm "No further splits...") ----
lgbm_param_dist = {
    "model__n_estimators": randint(400, 2000),
    "model__learning_rate": loguniform(3e-3, 1e-1),
    "model__num_leaves": randint(16, 255),
    "model__max_depth": randint(2, 12),

    "model__min_child_samples": randint(5, 80),
    "model__min_child_weight": loguniform(1e-3, 10.0),

    "model__subsample": uniform(0.6, 0.4),
    "model__subsample_freq": randint(0, 10),
    "model__colsample_bytree": uniform(0.6, 0.4),

    "model__reg_alpha": loguniform(1e-8, 1e-1),
    "model__reg_lambda": loguniform(1e-8, 10.0),

    "model__scale_pos_weight": [1.0, spw*0.75, spw, spw*1.25],
}

# ---- tune: ưu tiên ROC-AUC ----
search_lgbm = RandomizedSearchCV(
    estimator=lgbm_pipe,
    param_distributions=lgbm_param_dist,
    n_iter=LGBM_N_ITER,
    scoring={"roc_auc": "roc_auc", "pr_auc": "average_precision"},
    refit="roc_auc",
    cv=cv_lgbm,
    n_jobs=-1,
    random_state=SEED,
    verbose=2,
    return_train_score=False
)

search_lgbm.fit(X_train, y_train)

print(f"\nTotal candidates tried = {LGBM_N_ITER} | Total fits ≈ {LGBM_N_ITER * cv_lgbm.get_n_splits()}")
print("Best CV: ROC-AUC =", round(float(search_lgbm.best_score_), 5))
print("Best params:\n", search_lgbm.best_params_)

cvres = pd.DataFrame(search_lgbm.cv_results_)
best_idx = int(search_lgbm.best_index_)
if "mean_test_pr_auc" in cvres.columns:
    print("PR-AUC of best-ROC-AUC config =", round(float(cvres.loc[best_idx, "mean_test_pr_auc"]), 5))

print("\nTop 10 configs (by ROC-AUC):")
show_top_configs(search_lgbm, topk=10, metric="roc_auc")

print("\nTop 10 configs (by PR-AUC):")
show_top_configs(search_lgbm, topk=10, metric="pr_auc")

# ---- OOF + threshold + report ----
best_pipe_lgbm = search_lgbm.best_estimator_
oof_lgbm = get_oof_scores(best_pipe_lgbm, X_train, y_train, cv_lgbm)

thr_lgbm = pick_threshold_by_target_recall(
    y_train, oof_lgbm,
    target_recall=0.85,
    min_precision=0.30
)

m_lgbm, cm_lgbm = compute_metrics(y_train, oof_lgbm, thr_lgbm)

print(f"\nChosen threshold = {thr_lgbm:.4f}")
print(f"OOF ROC-AUC = {m_lgbm.get('roc_auc', np.nan):.4f} | PR-AUC = {m_lgbm.get('pr_auc', np.nan):.4f}")

qs = np.array([0.01, 0.05, 0.10, 0.25, 0.50])
print("Score quantiles:", np.quantile(oof_lgbm, qs))

# ---- confusion extraction fallback (nếu compute_metrics không trả dict) ----
tp = m_lgbm.get("TP", None); fp = m_lgbm.get("FP", None); tn = m_lgbm.get("TN", None); fn = m_lgbm.get("FN", None)
if any(v is None for v in [tp, fp, tn, fn]):
    if isinstance(cm_lgbm, dict):
        tp = cm_lgbm.get("TP", tp); fp = cm_lgbm.get("FP", fp); tn = cm_lgbm.get("TN", tn); fn = cm_lgbm.get("FN", fn)
    else:
        try:
            # sklearn confusion_matrix format: [[TN, FP],[FN, TP]]
            tn, fp, fn, tp = cm_lgbm.ravel()
        except Exception:
            pass

print("\nOOF metrics + confusion:")
cols = ["roc_auc","pr_auc","recall","precision","f1","specificity","bal_acc","brier","TP","FP","TN","FN"]
row = {
    "roc_auc": m_lgbm.get("roc_auc", np.nan),
    "pr_auc": m_lgbm.get("pr_auc", np.nan),
    "recall": m_lgbm.get("recall", np.nan),
    "precision": m_lgbm.get("precision", np.nan),
    "f1": m_lgbm.get("f1", np.nan),
    "specificity": m_lgbm.get("specificity", np.nan),
    "bal_acc": m_lgbm.get("bal_acc", np.nan),
    "brier": m_lgbm.get("brier", np.nan),
    "TP": tp, "FP": fp, "TN": tn, "FN": fn
}
display(pd.DataFrame([row], columns=cols).round(4))


Fitting 5 folds for each of 80 candidates, totalling 400 fits

Total candidates tried = 80 | Total fits ≈ 400
Best CV: ROC-AUC = 0.919
Best params:
 {'model__colsample_bytree': np.float64(0.6463476238100518), 'model__learning_rate': np.float64(0.061876126258211095), 'model__max_depth': 9, 'model__min_child_samples': 16, 'model__min_child_weight': np.float64(0.021066486017042207), 'model__n_estimators': 1471, 'model__num_leaves': 166, 'model__reg_alpha': np.float64(0.0004807162118091464), 'model__reg_lambda': np.float64(0.009846938138527219), 'model__scale_pos_weight': 6.896056661562021, 'model__subsample': np.float64(0.8550229885420852), 'model__subsample_freq': 2}
PR-AUC of best-ROC-AUC config = 0.82422

Top 10 configs (by ROC-AUC):


,rank_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,model__colsample_bytree,model__learning_rate,model__max_depth,model__min_child_samples,model__min_child_weight,model__n_estimators,model__num_leaves,model__reg_alpha,model__reg_lambda,model__scale_pos_weight,model__subsample,model__subsample_freq
0,1,0.91900,0.00589,0.64635,0.06188,9,16,0.02107,1471,166,0.00048,0.00985,6.89606,0.85502,2
1,2,0.90640,0.00658,0.90460,0.02622,7,11,0.05063,1591,185,0.00001,1.06432,4.13763,0.63254,4
2,3,0.90632,0.00860,0.79393,0.03401,11,48,0.00947,1797,175,0.00004,0.02124,1.00000,0.62596,9
3,4,0.90233,0.00632,0.88099,0.01058,11,30,1.73962,1320,167,0.02470,0.00040,1.00000,0.85116,3
4,5,0.90178,0.00723,0.87841,0.02214,9,36,0.48370,1361,142,0.00004,0.78769,1.00000,0.93947,8
5,6,0.90069,0.00943,0.87906,0.04898,11,60,2.33542,1184,99,0.00002,6.61140,1.00000,0.73897,2
6,7,0.90067,0.00729,0.99156,0.01653,9,31,0.05465,1700,45,0.00033,0.01048,1.00000,0.67810,2
7,8,0.90060,0.00490,0.89768,0.03268,10,67,0.08144,1832,71,0.00005,0.00008,4.13763,0.60486,4
8,9,0.89949,0.00479,0.78812,0.09435,7,15,1.84386,543,112,0.00000,0.00037,6.89606,0.61631,3
9,10,0.89941,0.00744,0.93508,0.03207,10,54,4.99775,1721,114,0.00000,0.00000,6.89606,0.82208,0



Top 10 configs (by PR-AUC):


,rank_test_pr_auc,mean_test_pr_auc,std_test_pr_auc,model__colsample_bytree,model__learning_rate,model__max_depth,model__min_child_samples,model__min_child_weight,model__n_estimators,model__num_leaves,model__reg_alpha,model__reg_lambda,model__scale_pos_weight,model__subsample,model__subsample_freq
0,1,0.82422,0.02017,0.64635,0.06188,9,16,0.02107,1471,166,0.00048,0.00985,6.89606,0.85502,2
1,2,0.79369,0.02502,0.79393,0.03401,11,48,0.00947,1797,175,0.00004,0.02124,1.00000,0.62596,9
2,3,0.79190,0.02700,0.90460,0.02622,7,11,0.05063,1591,185,0.00001,1.06432,4.13763,0.63254,4
3,4,0.78026,0.02021,0.78812,0.09435,7,15,1.84386,543,112,0.00000,0.00037,6.89606,0.61631,3
4,5,0.77303,0.01839,0.89768,0.03268,10,67,0.08144,1832,71,0.00005,0.00008,4.13763,0.60486,4
5,6,0.77186,0.02182,0.93508,0.03207,10,54,4.99775,1721,114,0.00000,0.00000,6.89606,0.82208,0
6,7,0.77084,0.01834,0.61736,0.07678,6,40,0.16422,1955,200,0.00005,0.00462,4.13763,0.71780,0
7,8,0.77071,0.02487,0.87841,0.02214,9,36,0.48370,1361,142,0.00004,0.78769,1.00000,0.93947,8
8,9,0.76999,0.02846,0.99156,0.01653,9,31,0.05465,1700,45,0.00033,0.01048,1.00000,0.67810,2
9,10,0.76928,0.02334,0.83406,0.03828,11,78,0.03246,1508,93,0.00000,0.00000,1.00000,0.84247,0



Chosen threshold = 0.0037
OOF ROC-AUC = 0.9188 | PR-AUC = 0.8237
Score quantiles: [4.71525624e-08 5.17962824e-07 2.11214178e-06 2.00417450e-05
 2.35036347e-04]

OOF metrics + confusion:


,roc_auc,pr_auc,recall,precision,f1,specificity,bal_acc,brier,TP,FP,TN,FN
0,0.9188,0.8237,0.8507,0.5062,0.6347,0.8495,0.8501,0.0582,1111,1084,6121,195


## Bảng tổng hợp sau khi train + tuning (OOF TRAIN)

**Giải thích các thông số:**
- **PR-AUC (Average Precision)**: phù hợp khi mất cân bằng lớp; càng cao càng tốt.
- **ROC-AUC**: khả năng phân tách 2 lớp; càng cao càng tốt.
- **Accuracy (ACC)**: tỷ lệ dự đoán đúng trên toàn bộ mẫu (**(TP + TN) / (TP + TN + FP + FN)**). *Lưu ý:* khi dữ liệu mất cân bằng, ACC có thể “ảo” (cao nhưng vẫn bỏ sót nhiều ca dương tính), nên nên xem kèm Recall/PR-AUC.
- **Recall / Sensitivity (TPR)**: ưu tiên y tế (bắt đúng ca CHD); càng cao càng tốt.
- **Precision (PPV)**: trong các ca dự đoán CHD=1, tỷ lệ đúng.
- **F1**: cân bằng giữa precision và recall.


In [71]:
# ============================================================
# OOF TRAIN EVAL (BEFORE tuning - BASELINE ONLY) — 5 models
# + Metrics: ROC-AUC, PR-AUC, ACC, Precision, Recall, F1, thr_used
# + In THAM SỐ MODEL ĐÃ TRAIN (baseline params) — gọn, dễ đọc
# ============================================================

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    precision_recall_curve,
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score
)

# 1) 5 model bạn muốn đánh giá
try:
    selected_models
except NameError:
    selected_models = ["ET", "HGB", "LGBM", "XGB", "RF"]

# 2) CV splitter
if "cv5" in globals():
    _cv = cv5
else:
    _cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED if "SEED" in globals() else 42)

# 3) CHỈ lấy baseline pipes (*_pipe_base)
_map_base = {
    "RF":  "rf_pipe_base",
    "ET":  "et_pipe_base",
    "HGB": "hgb_pipe_base",
    "XGB": "xgb_pipe_base",
    "LGBM":"lgbm_pipe_base",
}
baseline_pipes = {k: globals()[v] for k, v in _map_base.items() if v in globals()}

missing = [m for m in selected_models if m not in baseline_pipes]
if missing:
    raise NameError(f"Thiếu pipeline baseline cho: {missing}. "
                    f"Hãy chạy các cell tạo *_pipe_base (vd: et_pipe_base, rf_pipe_base, ...).")

# 4) threshold map từ baseline_results nếu có
thr_map = {}
if "baseline_results" in globals():
    _bdf = pd.DataFrame(baseline_results).copy()
    if "thr" in _bdf.columns:
        for c in ["f1","recall"]:
            if c in _bdf.columns:
                _bdf[c] = pd.to_numeric(_bdf[c], errors="coerce")
        if "f1" in _bdf.columns:
            _bdf = _bdf.sort_values("f1", ascending=False).drop_duplicates("model", keep="first")
        elif "recall" in _bdf.columns:
            _bdf = _bdf.sort_values("recall", ascending=False).drop_duplicates("model", keep="first")
        thr_map = _bdf.set_index("model")["thr"].to_dict()

def pick_thr_max_f1(y_true, scores):
    prec, rec, thr = precision_recall_curve(y_true, scores)
    if len(thr) == 0:
        return 0.5
    f1s = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
    return float(thr[np.argmax(f1s)])

# 5) Helpers lấy params gọn
def get_model_obj(pipe):
    if hasattr(pipe, "named_steps") and "model" in pipe.named_steps:
        return pipe.named_steps["model"]
    return None

def compact_params(model):
    # gọn theo từng loại model phổ biến
    cls = model.__class__.__name__.lower()
    p = model.get_params(deep=False)

    if "logistic" in cls:
        keys = ["C","penalty","solver","class_weight","max_iter"]
    elif "randomforest" in cls:
        keys = ["n_estimators","max_depth","min_samples_split","min_samples_leaf","max_features","class_weight","bootstrap"]
    elif "extra" in cls:
        keys = ["n_estimators","max_depth","min_samples_split","min_samples_leaf","max_features","bootstrap"]
    elif "histgradientboosting" in cls:
        keys = ["learning_rate","max_depth","max_iter","min_samples_leaf","l2_regularization","max_bins"]
    elif "xgb" in cls or "xgboost" in cls:
        keys = ["n_estimators","learning_rate","max_depth","subsample","colsample_bytree","reg_alpha","reg_lambda","min_child_weight","gamma","scale_pos_weight"]
    elif "lgbm" in cls or "lightgbm" in cls:
        keys = ["n_estimators","learning_rate","num_leaves","max_depth","min_child_samples",
                "subsample","subsample_freq","colsample_bytree","reg_alpha","reg_lambda","scale_pos_weight"]
    else:
        # fallback: lấy một số key hay gặp
        keys = ["n_estimators","max_depth","learning_rate","C","class_weight"]

    out = {"model_type": model.__class__.__name__}
    for k in keys:
        if k in p:
            out[k] = p[k]
    return out

# 6) Evaluate OOF + collect params
metric_rows = []
param_rows = []

for m in selected_models:
    pipe = baseline_pipes[m]
    oof_scores = get_oof_scores(pipe, X_train, y_train, _cv)

    thr = float(thr_map.get(m, np.nan))
    if not np.isfinite(thr):
        thr = pick_thr_max_f1(y_train, oof_scores)

    y_pred = (oof_scores >= thr).astype(int)

    metric_rows.append({
        "model": m,
        "thr_used": thr,
        "roc_auc": roc_auc_score(y_train, oof_scores),
        "pr_auc": average_precision_score(y_train, oof_scores),
        "acc": accuracy_score(y_train, y_pred),
        "precision": precision_score(y_train, y_pred, zero_division=0),
        "recall": recall_score(y_train, y_pred, zero_division=0),
        "f1": f1_score(y_train, y_pred, zero_division=0),
    })

    model_obj = get_model_obj(pipe)
    if model_obj is None:
        param_rows.append({"model": m, "note": "Không tìm thấy named_steps['model'] trong pipeline"})
    else:
        d = {"model": m}
        d.update(compact_params(model_obj))
        param_rows.append(d)

# 7) Display tables
report_before_oof = pd.DataFrame(metric_rows).sort_values("f1", ascending=False).reset_index(drop=True)
display(report_before_oof.round(4))

print("\nBaseline TRAINED PARAMS (compact):")
params_before = pd.DataFrame(param_rows)
display(params_before)


,model,thr_used,roc_auc,pr_auc,acc,precision,recall,f1
0,ET,0.3375,0.9462,0.8565,0.9356,0.8616,0.6914,0.7672
1,RF,0.2800,0.9381,0.8435,0.9299,0.8043,0.7175,0.7584
2,LGBM,0.2468,0.8879,0.7256,0.9041,0.7018,0.6524,0.6762
3,HGB,0.2682,0.8621,0.6490,0.8845,0.6308,0.5965,0.6131
4,XGB,0.2393,0.7772,0.4780,0.8221,0.4318,0.5046,0.4654



Baseline TRAINED PARAMS (compact):


,model,model_type,n_estimators,max_depth,min_samples_split,min_samples_leaf,max_features,bootstrap,learning_rate,max_iter,...,min_child_samples,subsample,subsample_freq,colsample_bytree,reg_alpha,reg_lambda,min_child_weight,gamma,scale_pos_weight,class_weight
0,ET,ExtraTreesClassifier,400.0,NaN,2.0,1.0,sqrt,False,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HGB,HistGradientBoostingClassifier,NaN,NaN,NaN,20.0,NaN,NaN,0.10,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LGBM,LGBMClassifier,400.0,-1.0,NaN,NaN,NaN,NaN,0.05,NaN,...,20.0,1.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN
3,XGB,XGBClassifier,400.0,3.0,NaN,NaN,NaN,NaN,0.05,NaN,...,NaN,0.9,NaN,0.9,NaN,NaN,NaN,NaN,NaN,NaN
4,RF,RandomForestClassifier,400.0,NaN,2.0,1.0,sqrt,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,balanced


In [72]:
# ============================================================
# IN THAM SỐ "MODEL ĐÃ TRAIN" (baseline & tuned) + threshold
# - KHÔNG in best_params_ (tuning)
# - Chỉ lấy từ object model đã fit / được dùng để predict:
#   + baseline: *_pipe_base (hoặc baseline_pipes nếu có)
#   + tuned: search_*.best_estimator_
# ============================================================

import pandas as pd
from pprint import pprint

# 1) 5 model bạn muốn in
try:
    selected_models
except NameError:
    selected_models = ["ET", "HGB", "LGBM", "XGB", "RF"]

# 2) Map threshold (nếu có)
def _thr_map(results):
    df = pd.DataFrame(results).copy()
    if df.empty or "model" not in df.columns:
        return {}
    thr_col = "thr" if "thr" in df.columns else None
    if not thr_col:
        return {}
    # nếu trùng model, giữ dòng tốt nhất theo f1 rồi recall
    for c in ["f1", "recall"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    if "f1" in df.columns:
        df = df.sort_values("f1", ascending=False)
    elif "recall" in df.columns:
        df = df.sort_values("recall", ascending=False)
    df = df.drop_duplicates("model", keep="first")
    return df.set_index("model")[thr_col].to_dict()

thr_before = _thr_map(baseline_results) if "baseline_results" in globals() else {}
thr_after  = _thr_map(tuned_results)    if "tuned_results" in globals() else {}

# 3) Lấy baseline pipes (nếu bạn chưa có dict baseline_pipes thì auto build từ *_pipe_base)
if "baseline_pipes" not in globals():
    baseline_pipes = {}
    for name, var in {
        "RF": "rf_pipe_base",
        "ET": "et_pipe_base",
        "HGB": "hgb_pipe_base",
        "XGB": "xgb_pipe_base",
        "LGBM":"lgbm_pipe_base",
    }.items():
        if var in globals():
            baseline_pipes[name] = globals()[var]

# 4) Map search objects (tuned)
search_map = {
    "RF":  "search_rf",
    "ET":  "search_et",
    "HGB": "search_hgb",
    "XGB": "search_xgb",
    "LGBM":"search_lgbm",
}

def _get_model_params_from_pipe(pipe, deep=False):
    if hasattr(pipe, "named_steps") and "model" in pipe.named_steps:
        return pipe.named_steps["model"].get_params(deep=deep)
    # fallback (hiếm)
    return pipe.get_params(deep=deep) if hasattr(pipe, "get_params") else {}

# 5) In tham số train (không in best_params_)
for m in selected_models:
    print("\n" + "="*90)
    print(f"MODEL: {m}")

    # ---- BASELINE trained params ----
    if m in baseline_pipes:
        base_params = _get_model_params_from_pipe(baseline_pipes[m], deep=False)
        print(f"[BASELINE] thr = {thr_before.get(m, None)}")
        print(f"[BASELINE] trained model params (get_params(deep=False)):")
        pprint(base_params)
    else:
        print("[BASELINE] Không tìm thấy pipeline baseline (vd: et_pipe_base/rf_pipe_base...).")

    # ---- TUNED trained params (best_estimator_) ----
    s_var = search_map.get(m)
    if s_var in globals():
        s = globals()[s_var]
        tuned_pipe = s.best_estimator_
        tuned_params = _get_model_params_from_pipe(tuned_pipe, deep=False)
        print(f"\n[TUNED] thr = {thr_after.get(m, None)}")
        print("[TUNED] trained model params (best_estimator_.named_steps['model'].get_params(deep=False)):")
        pprint(tuned_params)
    else:
        print("\n[TUNED] Không tìm thấy search object (vd: search_et/search_lgbm...).")



MODEL: ET
[BASELINE] thr = 0.3375
[BASELINE] trained model params (get_params(deep=False)):
{'bootstrap': False,
 'ccp_alpha': 0.0,
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 400,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

[TUNED] thr = 0.18009127949386214
[TUNED] trained model params (best_estimator_.named_steps['model'].get_params(deep=False)):
{'bootstrap': False,
 'ccp_alpha': 0.0,
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': 30,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 659,
 '

In [73]:
# ============================================================
# OOF TRAIN EVAL (AFTER tuning) — 5 models
# Metrics: ACC, Precision, Recall, F1 + ROC-AUC + PR-AUC
# + IN RA "THÔNG SỐ MODEL ĐÃ TRAIN" (tuned params) + threshold đã dùng
# ============================================================

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    precision_recall_curve,
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score
)
from pprint import pprint

# 1) 5 model bạn muốn đánh giá
try:
    selected_models
except NameError:
    selected_models = ["ET", "HGB", "LGBM", "XGB", "RF"]

# 2) CV splitter
if "cv5" in globals():
    _cv = cv5
else:
    _cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED if "SEED" in globals() else 42)

# 3) Auto-build tuned_pipes nếu chưa có (ưu tiên search_* => best_estimator_)
if "tuned_pipes" not in globals():
    _tuned_map = {
        "RF":  ["search_rf",  "best_pipe_rf",  "rf_pipe_tuned",  "rf_pipe_best"],
        "ET":  ["search_et",  "best_pipe_et",  "et_pipe_tuned",  "et_pipe_best"],
        "HGB": ["search_hgb", "best_pipe_hgb", "hgb_pipe_tuned", "hgb_pipe_best"],
        "XGB": ["search_xgb", "best_pipe_xgb", "xgb_pipe_tuned", "xgb_pipe_best"],
        "LGBM":["search_lgbm","best_pipe_lgbm","lgbm_pipe_tuned","lgbm_pipe_best"],
    }
    tuned_pipes = {}
    for k, cand_vars in _tuned_map.items():
        for v in cand_vars:
            if v in globals():
                obj = globals()[v]
                tuned_pipes[k] = obj.best_estimator_ if hasattr(obj, "best_estimator_") else obj
                break

# 4) Threshold map từ tuned_results nếu có
thr_map = {}
if "tuned_results" in globals():
    _tdf = pd.DataFrame(tuned_results).copy()
    if "thr" in _tdf.columns:
        for c in ["f1", "recall"]:
            if c in _tdf.columns:
                _tdf[c] = pd.to_numeric(_tdf[c], errors="coerce")
        if "f1" in _tdf.columns:
            _tdf = _tdf.sort_values("f1", ascending=False).drop_duplicates("model", keep="first")
        elif "recall" in _tdf.columns:
            _tdf = _tdf.sort_values("recall", ascending=False).drop_duplicates("model", keep="first")
        thr_map = _tdf.set_index("model")["thr"].to_dict()

def pick_thr_max_f1(y_true, scores):
    prec, rec, thr = precision_recall_curve(y_true, scores)
    if len(thr) == 0:
        return 0.5
    f1s = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
    return float(thr[np.argmax(f1s)])

# 5) Evaluate OOF + IN PARAMS
missing = [m for m in selected_models if m not in tuned_pipes]
if missing:
    raise NameError(
        f"Thiếu pipeline TUNED cho: {missing}. "
        f"Hãy chắc chắn đã chạy tuning để có search_* hoặc best_pipe_* (vd: search_lgbm, best_pipe_lgbm, ...)."
    )

rows = []
param_rows = []

for m in selected_models:
    pipe = tuned_pipes[m]
    oof_scores = get_oof_scores(pipe, X_train, y_train, _cv)

    thr = float(thr_map.get(m, np.nan))
    if not np.isfinite(thr):
        thr = pick_thr_max_f1(y_train, oof_scores)

    y_pred = (oof_scores >= thr).astype(int)

    rows.append({
        "model": m,
        "thr_used": thr,
        "roc_auc": roc_auc_score(y_train, oof_scores),
        "pr_auc": average_precision_score(y_train, oof_scores),
        "acc": accuracy_score(y_train, y_pred),
        "precision": precision_score(y_train, y_pred, zero_division=0),
        "recall": recall_score(y_train, y_pred, zero_division=0),
        "f1": f1_score(y_train, y_pred, zero_division=0),
    })

    # params "model đã train" (không in best_params_)
    if hasattr(pipe, "named_steps") and "model" in pipe.named_steps:
        params = pipe.named_steps["model"].get_params(deep=False)
    else:
        params = pipe.get_params(deep=False) if hasattr(pipe, "get_params") else {}

    param_rows.append({"model": m, "thr_used": thr, "trained_params": params})

report_after_oof = pd.DataFrame(rows).sort_values("f1", ascending=False).reset_index(drop=True)
display(report_after_oof.round(4))

print("\n" + "="*90)
print("TUNED TRAINED PARAMS (mỗi model):")
for r in param_rows:
    print("\n" + "-"*90)
    print(f"MODEL: {r['model']} | thr_used: {r['thr_used']:.6f}")
    pprint(r["trained_params"])


,model,thr_used,roc_auc,pr_auc,acc,precision,recall,f1
0,LGBM,0.1641,0.9188,0.8237,0.9331,0.8264,0.7144,0.7663
1,RF,0.2768,0.9388,0.8463,0.9300,0.8024,0.7213,0.7597
2,ET,0.1801,0.9456,0.8565,0.8850,0.5863,0.8507,0.6942
3,HGB,0.0021,0.9253,0.8316,0.8697,0.5486,0.8507,0.6671
4,XGB,0.0448,0.8862,0.7248,0.7421,0.3571,0.8507,0.5031



TUNED TRAINED PARAMS (mỗi model):

------------------------------------------------------------------------------------------
MODEL: ET | thr_used: 0.180091
{'bootstrap': False,
 'ccp_alpha': 0.0,
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': 30,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 659,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

------------------------------------------------------------------------------------------
MODEL: HGB | thr_used: 0.002144
{'categorical_features': 'from_dtype',
 'class_weight': None,
 'early_stopping': 'auto',
 'interaction_cst': None,
 'l2_regularization': np.float64(0.0022057229453746087),
 'learning_rate': np.float64(0.07190034331426896),
 'loss': 'log_loss',
 'max_bins': 163,
 'max_depth': None,
 

## Best after tuning + đánh giá cuối trên TEST (chỉ 1 lần)



In [74]:
# ==============================
# RF (TUNED) -> TEST (no plot)
# ==============================
import numpy as np
import pandas as pd
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix
)

# Get tuned RF pipeline
if "search_rf" in globals():
    rf_pipe = search_rf.best_estimator_
elif "tuned_pipes" in globals() and "RF" in tuned_pipes:
    rf_pipe = tuned_pipes["RF"]
elif "rf_pipe_base" in globals():
    print("⚠️ No tuned RF found -> using baseline rf_pipe_base.")
    rf_pipe = rf_pipe_base
else:
    raise NameError("❌ RF pipeline not found. Run RF build/tuning cells first (search_rf or tuned_pipes).")

# Get threshold used
rf_thr = None
if "report_after_oof" in globals():
    try:
        rf_thr = float(report_after_oof.loc[report_after_oof["model"] == "RF", "thr_used"].iloc[0])
    except Exception:
        rf_thr = None
if rf_thr is None and "thr_map" in globals() and "RF" in thr_map:
    rf_thr = float(thr_map["RF"])
if rf_thr is None and "rf_thr" in globals():
    rf_thr = float(rf_thr)
if rf_thr is None:
    rf_thr = 0.5

# Fit and predict
rf_pipe.fit(X_train, y_train)
proba = rf_pipe.predict_proba(X_test)[:, 1]
pred  = (proba >= rf_thr).astype(int)

# Metrics
roc  = roc_auc_score(y_test, proba)
pra  = average_precision_score(y_test, proba)
acc  = accuracy_score(y_test, pred)
pre  = precision_score(y_test, pred, zero_division=0)
rec  = recall_score(y_test, pred, zero_division=0)
f1   = f1_score(y_test, pred, zero_division=0)
cm   = confusion_matrix(y_test, pred)

print(f"[RF TEST] thr={rf_thr:.4f} | AUC={roc:.4f} | PR-AUC={pra:.4f} | ACC={acc:.4f} | P={pre:.4f} | R={rec:.4f} | F1={f1:.4f}")
display(pd.DataFrame(cm, index=["True0","True1"], columns=["Pred0","Pred1"]))


[RF TEST] thr=0.2768 | AUC=0.9655 | PR-AUC=0.9040 | ACC=0.9445 | P=0.8276 | R=0.8073 | F1=0.8173


,Pred0,Pred1
True0,1746,55
True1,63,264


In [ ]:
# ==============================
# ET (TUNED) -> TEST (no plot)
# ==============================
import numpy as np
import pandas as pd
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix
)

# Get tuned ET pipeline
if "search_et" in globals():
    et_pipe = search_et.best_estimator_
elif "search_ET" in globals():
    et_pipe = search_ET.best_estimator_
elif "tuned_pipes" in globals() and "ET" in tuned_pipes:
    et_pipe = tuned_pipes["ET"]
elif "et_pipe_base" in globals():
    print("⚠️ No tuned ET found -> using baseline et_pipe_base.")
    et_pipe = et_pipe_base
else:
    raise NameError("❌ ET pipeline not found. Run ET build/tuning cells first (search_et or tuned_pipes).")

# Get threshold used
et_thr = None
if "report_after_oof" in globals():
    try:
        et_thr = float(report_after_oof.loc[report_after_oof["model"] == "ET", "thr_used"].iloc[0])
    except Exception:
        et_thr = None
if et_thr is None and "thr_map" in globals() and "ET" in thr_map:
    et_thr = float(thr_map["ET"])
if et_thr is None and "et_thr" in globals():
    et_thr = float(et_thr)
if et_thr is None:
    et_thr = 0.5

# Fit and predict
et_pipe.fit(X_train, y_train)
proba = et_pipe.predict_proba(X_test)[:, 1]
pred  = (proba >= et_thr).astype(int)

# Metrics
roc  = roc_auc_score(y_test, proba)
pra  = average_precision_score(y_test, proba)
acc  = accuracy_score(y_test, pred)
pre  = precision_score(y_test, pred, zero_division=0)
rec  = recall_score(y_test, pred, zero_division=0)
f1   = f1_score(y_test, pred, zero_division=0)
cm   = confusion_matrix(y_test, pred)    

print(f"[ET TEST] thr={et_thr:.4f} | AUC={roc:.4f} | PR-AUC={pra:.4f} | ACC={acc:.4f} | P={pre:.4f} | R={rec:.4f} | F1={f1:.4f}")
display(pd.DataFrame(cm, index=["True0","True1"], columns=["Pred0","Pred1"]))


[ET TEST] thr=0.1801 | AUC=0.9705 | PR-AUC=0.9114 | ACC=0.9023 | P=0.6258 | R=0.9052 | F1=0.7400


,Pred0,Pred1
True0,1624,177
True1,31,296


In [ ]:
# # ==============================
# # SAVE 2 TUNED MODELS: RF + ET -> đúng thư mục chứa notebook (Windows path)
# # ==============================
# import os
# import joblib
# import numpy as np
# import pandas as pd

# # ---- SET NOTEBOOK PATH (Windows) ----
# NOTEBOOK_PATH = r"E:\Antigravity\124231\ML\Heart Disease Prediction Dataset\CHD_10Y_D11KS.ipynb"

# # Lưu model vào cùng folder với notebook, trong subfolder "saved_models"
# out_dir = os.path.join(os.path.dirname(NOTEBOOK_PATH), "saved_models")
# os.makedirs(out_dir, exist_ok=True)

# # ---------- helpers ----------
# def _get_pipe(model_key: str):
#     key = model_key.upper()
#     if key == "RF":
#         if "search_rf" in globals(): return search_rf.best_estimator_
#         if "tuned_pipes" in globals() and "RF" in tuned_pipes: return tuned_pipes["RF"]
#         if "rf_pipe_base" in globals():
#             print("⚠️ RF tuned not found -> using baseline rf_pipe_base")
#             return rf_pipe_base
#         raise NameError("❌ RF pipeline not found (search_rf / tuned_pipes['RF'] / rf_pipe_base).")
#     if key == "ET":
#         if "search_et" in globals(): return search_et.best_estimator_
#         if "search_ET" in globals(): return search_ET.best_estimator_
#         if "tuned_pipes" in globals() and "ET" in tuned_pipes: return tuned_pipes["ET"]
#         if "et_pipe_base" in globals():
#             print("⚠️ ET tuned not found -> using baseline et_pipe_base")
#             return et_pipe_base
#         raise NameError("❌ ET pipeline not found (search_et / tuned_pipes['ET'] / et_pipe_base).")
#     raise ValueError("model_key must be 'RF' or 'ET'")

# def _get_thr(model_key: str, default=0.5):
#     key = model_key.upper()
#     thr = None
#     if "report_after_oof" in globals():
#         try:
#             thr = float(report_after_oof.loc[report_after_oof["model"] == key, "thr_used"].iloc[0])
#         except Exception:
#             thr = None
#     if thr is None and "thr_map" in globals() and key in thr_map:
#         thr = float(thr_map[key])
#     if thr is None:
#         if key == "RF" and "rf_thr" in globals(): thr = float(rf_thr)
#         if key == "ET" and "et_thr" in globals(): thr = float(et_thr)
#     return float(thr) if thr is not None else float(default)

# def _get_full_data():
#     if "X" in globals() and "y" in globals():
#         return X, y
#     if "X_train" in globals() and "y_train" in globals() and "X_test" in globals() and "y_test" in globals():
#         X_full = pd.concat([X_train, X_test], axis=0)
#         y_full = pd.concat([y_train, y_test], axis=0)
#         return X_full, y_full
#     if "X_train" in globals() and "y_train" in globals():
#         return X_train, y_train
#     raise NameError("❌ Không tìm thấy data (X,y) hoặc (X_train,y_train).")

# def _feature_names(X_):
#     try:
#         return list(X_.columns)
#     except Exception:
#         return None

# # ---------- get models & thresholds ----------
# rf_pipe = _get_pipe("RF")
# et_pipe = _get_pipe("ET")
# rf_thr  = _get_thr("RF", default=0.5)
# et_thr  = _get_thr("ET", default=0.5)

# # ---------- fit on full data for deployment ----------
# X_full, y_full = _get_full_data()
# rf_pipe.fit(X_full, y_full)
# et_pipe.fit(X_full, y_full)

# # ---------- save ----------
# rf_path = os.path.join(out_dir, "CHD_RF_tuned.joblib")
# et_path = os.path.join(out_dir, "CHD_ET_tuned.joblib")

# joblib.dump({"name":"RF","pipeline":rf_pipe,"threshold":rf_thr,"feature_names":_feature_names(X_full)}, rf_path)
# joblib.dump({"name":"ET","pipeline":et_pipe,"threshold":et_thr,"feature_names":_feature_names(X_full)}, et_path)

# print("✅ Saved to:")
# print(" -", rf_path, f"(thr={rf_thr:.4f})")
# print(" -", et_path, f"(thr={et_thr:.4f})")


✅ Saved to:
 - E:\Antigravity\124231\ML\Heart Disease Prediction Dataset\saved_models\CHD_RF_tuned.joblib (thr=0.2768)
 - E:\Antigravity\124231\ML\Heart Disease Prediction Dataset\saved_models\CHD_ET_tuned.joblib (thr=0.1801)


In [84]:
import numpy as np
import pandas as pd
import plotly.express as px

def get_feature_names_pipeline(pipe, X):
    # ưu tiên lấy tên feature sau preprocess
    if hasattr(pipe, "named_steps"):
        # thử các step có get_feature_names_out
        for _, step in pipe.named_steps.items():
            if hasattr(step, "get_feature_names_out"):
                try:
                    return list(step.get_feature_names_out())
                except Exception:
                    pass
        # thử phần preprocess (cắt estimator cuối)
        try:
            pre = pipe[:-1]
            if hasattr(pre, "get_feature_names_out"):
                return list(pre.get_feature_names_out())
        except Exception:
            pass

    return list(getattr(X, "columns", [f"f{i}" for i in range(X.shape[1])]))

def strip_transformer_prefix(s):
    # ví dụ: "num__age" -> "age", "cat__gender_M" -> "gender_M"
    return s.split("__", 1)[-1] if isinstance(s, str) else s

def base_feature_name(transformed_name, original_cols):
    """
    Gộp one-hot: 'gender_M' -> 'gender' (nếu 'gender' là cột gốc).
    Làm kiểu "match prefix theo cột gốc dài nhất".
    """
    s = transformed_name
    best = None
    for col in original_cols:
        if s == col or s.startswith(col + "_"):
            if best is None or len(col) > len(best):
                best = col
    return best if best is not None else s

def plot_et_feature_importance_eda_style(et_pipe, X_ref, top_n=20, aggregate=True, title=None):
    # estimator cuối
    est = et_pipe.steps[-1][1] if hasattr(et_pipe, "steps") else et_pipe
    if not hasattr(est, "feature_importances_"):
        raise ValueError("ET phải có feature_importances_. Check lại et_pipe có đúng ExtraTrees không.")

    names = get_feature_names_pipeline(et_pipe, X_ref)
    imp = np.array(est.feature_importances_, dtype=float)

    n = min(len(names), len(imp))
    names, imp = names[:n], imp[:n]

    df = pd.DataFrame({"Feature": names, "Importance": imp})
    df["Feature"] = df["Feature"].map(strip_transformer_prefix)

    if aggregate:
        orig_cols = list(getattr(X_ref, "columns", []))
        if len(orig_cols) > 0:
            df["BaseFeature"] = df["Feature"].apply(lambda s: base_feature_name(s, orig_cols))
        else:
            df["BaseFeature"] = df["Feature"].apply(lambda s: s.split("_")[0] if "_" in s else s)

        df_plot = (df.groupby("BaseFeature", as_index=False)["Importance"]
                     .sum()
                     .sort_values("Importance", ascending=True)
                     .tail(top_n))
        ycol = "BaseFeature"
    else:
        df_plot = (df.sort_values("Importance", ascending=True)
                     .tail(top_n))
        ycol = "Feature"

    fig = px.bar(
        df_plot,
        x="Importance",
        y=ycol,
        orientation="h",
        text="Importance",
        title=title or (f"ET Feature Importance ")
    )
    fig.update_traces(texttemplate="%{text:.4f}", textposition="outside", cliponaxis=False)
    fig.update_layout(height=max(420, 24*top_n + 140), margin=dict(l=120, r=30, t=70, b=20))
    fig.show()

    return df

# === DÙNG NGAY ===
# ET đã fit rồi thì gọi:
_ = plot_et_feature_importance_eda_style(
    et_pipe,
    X_test,        # lấy tên cột gốc từ đây cho chuẩn
    top_n=20,
    aggregate=True # gộp one-hot thành feature gốc cho gọn
)

# Nếu bạn muốn xem chi tiết từng cột one-hot (không gộp):
# _ = plot_et_feature_importance_eda_style(et_pipe, X_test, top_n=30, aggregate=False)
